# Etude d'une perturbation barocline dans les analyses ECMWF

Auteur : FERRY Frédéric (DESR/ENM/C3M) - novembre 2022

<img src="201401260600.png" width="800">


Données issues des analyses ECMWF :
https://www.ecmwf.int/en/forecasts/dataset/operational-archive

Concepts illustrés :
- Calcul d'épaisseurs
- Calcul de la témpérature potentielle
- Calcul de la température potentielle équivalente (fonction Metpy)
- Calcul de l'humidité relative (fonction Metpy)
- Calcul du vent géostrophique/agéostrophique
- Calcul du vent thermique
- Calcul de la divergence
- Calcul du tourbillon (relatif/absolu)
- Calcul du tourbillon potentiel d'Ertel
- Calcul d'advection de température
- Calcul d'advection différentielle de tourbillon
- Tracés de cartes et de coupes verticales
- Ajout des min/max sur une carte
- Découpage d'un cercle autour d'un système

Concepts avancés de météorologie dynamique illustrés :
- Filtrage spatial de champs pour isoler les structures de grande échelle (fonction gaussian_filter de scipy)
- Interpolation d'un champ sur une surface isentrope à 𝛉 constante (fonction Metpy)
- Calcul du forçage géostrophique (Q1, Q2, div(Q))
- Fonction de frontogenèse et champ de déformation

In [ ]:
import os

import xarray as xr
import numpy as np

import cartopy.crs as ccrs
from cartopy.mpl.geoaxes import GeoAxes
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
import cartopy.feature as cfeature
from cartopy.mpl.patch import geos_to_path

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.path as mpath
from mpl_toolkits.axes_grid1 import AxesGrid
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from matplotlib.collections import LineCollection

import metpy
import metpy.calc as mpcalc
from metpy.units import units

from scipy.ndimage import gaussian_filter
from scipy.ndimage import maximum_filter, minimum_filter

import IPython.display as IPdisplay, matplotlib.font_manager as fm
from PIL import Image
import glob
import itertools

import warnings
warnings.filterwarnings('ignore')

In [ ]:
re=6.37e6
g=9.81
omega=7.292e-5

cp=1004.5
r=2*cp/7
kap=r/cp

# Ouverture des données netcdf (z, u, v, t, pv, w, q)

In [ ]:
date1='20140125-T00'
date2='20140126-T18'

In [ ]:
fp    = xr.open_dataset('./ecmwf/20140124-26_msl.nc').sel(time=slice(date1,date2))
fz    = xr.open_dataset('./ecmwf/20140124-26_z.nc').sel(time=slice(date1,date2))
fu    = xr.open_dataset('./ecmwf/20140124-26_u.nc').sel(time=slice(date1,date2))
fv    = xr.open_dataset('./ecmwf/20140124-26_v.nc').sel(time=slice(date1,date2))
ft    = xr.open_dataset('./ecmwf/20140124-26_t.nc').sel(time=slice(date1,date2))
fpv    = xr.open_dataset('./ecmwf/20140124-26_pv.nc').sel(time=slice(date1,date2))
fw    = xr.open_dataset('./ecmwf/20140124-26_w.nc').sel(time=slice(date1,date2))
fq    = xr.open_dataset('./ecmwf/20140124-26_q.nc').sel(time=slice(date1,date2))

print(fq.variables)

msl0 = fp['msl']/100
z0 = fz['z']/g
t0 = ft['t']
u0 = fu['u']
v0 = fv['v']
w0 = fw['w']
q0 = fq['q']
pv0 = fpv['pv']

print(pv0.shape)
print(msl0.shape)

In [ ]:
time  = fz.time.values

time_str=[x for x in range(len(time))]
date_str=[x for x in range(len(time))]
for i in range(len(time)):
	time_str[i] = str(time[i])
	date_str[i] = time_str[i][0:13]
print(date_str)

In [ ]:
lev = fz.level.values
print(lev)

lev925 = list(lev).index(925)
lev850 = list(lev).index(850)
lev700 = list(lev).index(700)
lev600 = list(lev).index(600)
lev500 = list(lev).index(500)
lev300 = list(lev).index(300)

In [ ]:
def lonflip(da):
    lon_name = 'longitude'
    da['_longitude_adjusted'] = xr.where(
        da[lon_name] > 180,
        da[lon_name] - 360,
        da[lon_name])
    da = (
        da
        .swap_dims({lon_name: '_longitude_adjusted'})
        .sel(**{'_longitude_adjusted': sorted(da._longitude_adjusted)})
        .drop(lon_name))
    da = da.rename({'_longitude_adjusted': lon_name})
    return da

In [ ]:
z1 = lonflip(z0)
u1 = lonflip(u0)
v1 = lonflip(v0)
t1 = lonflip(t0)
w1 = lonflip(w0)

msl1 = lonflip(msl0)
z1 = lonflip(z0)
t1 = lonflip(t0)
u1 = lonflip(u0)
v1 = lonflip(v0)
w1 = lonflip(w0)
q1 = lonflip(q0)
pv1 = lonflip(pv0)

lon0  = w1.longitude.values
print(lon0)

In [ ]:
latS=35
latN=65
lonW=-50
lonE=20

msl = msl1.sel(latitude=slice(latN,latS)).sel(longitude=slice(lonW,lonE))
z = z1.sel(latitude=slice(latN,latS)).sel(longitude=slice(lonW,lonE))
u = u1.sel(latitude=slice(latN,latS)).sel(longitude=slice(lonW,lonE))
v = v1.sel(latitude=slice(latN,latS)).sel(longitude=slice(lonW,lonE))
t = t1.sel(latitude=slice(latN,latS)).sel(longitude=slice(lonW,lonE))
w = w1.sel(latitude=slice(latN,latS)).sel(longitude=slice(lonW,lonE))
q = q1.sel(latitude=slice(latN,latS)).sel(longitude=slice(lonW,lonE))
pv = pv1.sel(latitude=slice(latN,latS)).sel(longitude=slice(lonW,lonE))

lat = z.latitude.values
lon  = w.longitude.values

print(z.shape)
print(u.shape)
print(v.shape)
print(w.shape)
print(t.shape)
print(q.shape)
print(pv.shape)
print(msl.shape)

# Diagnostics : module du vent, épaisseurs

In [ ]:
uv_mod=np.sqrt(u**2+v**2)
print(np.min(uv_mod[:,lev300,:,:]))
print(np.max(uv_mod[:,lev300,:,:]))

In [ ]:
dz=z.sel(level=500)-z.sel(level=925)
print(np.min(dz))
print(np.max(dz))

# Diagnostics : theta, thetae, humidité relative

In [ ]:
p=lev[np.newaxis,:,np.newaxis,np.newaxis]
print(p.shape)

$$\theta = \left(\frac{P_0}{P}\right)^\frac{R}{C_P}$$

In [ ]:
theta=t*(1.E5/(p*100))**kap
print(theta.shape)
print(np.min(theta[:,lev850,:,:]))
print(np.max(theta[:,lev850,:,:]))

https://unidata.github.io/MetPy/latest/api/generated/metpy.calc.dewpoint_from_specific_humidity.html

In [ ]:
#Td = mpcalc.dewpoint_from_specific_humidity(q.values* units('kg/kg'), t.values* units.kelvin, p* units.hPa) # old Metpy
Td = mpcalc.dewpoint_from_specific_humidity(p* units.hPa, t.values* units.kelvin, q.values* units('kg/kg'))

https://unidata.github.io/MetPy/latest/api/generated/metpy.calc.relative_humidity_from_specific_humidity.html

In [ ]:
rh=mpcalc.relative_humidity_from_specific_humidity(p* units.hPa, t.values* units.kelvin, q.values* units('kg/kg'))

https://unidata.github.io/MetPy/latest/api/generated/metpy.calc.equivalent_potential_temperature.html

In [ ]:
thetae=mpcalc.equivalent_potential_temperature(p* units.hPa, t, Td)
print(thetae.shape)
print(np.min(thetae[:,lev850,:,:]))
print(np.max(thetae[:,lev850,:,:]))

# Diagnostics : vent géostrophique, vent agéostrophique, vent thermique

$$dx = a cos(\phi)d\lambda$$
$$dy = a d\phi$$

In [ ]:
xlon,ylat=np.meshgrid(lon,lat)
dlony,dlonx=np.gradient(xlon)
dlaty,dlatx=np.gradient(ylat)
dx=re*np.cos(ylat*np.pi/180)*dlonx*np.pi/180
dy=re*dlaty*np.pi/180
print(dx.shape)
print(dy.shape)

$$f = 2\Omega sin(\phi)$$

In [ ]:
f=2*omega*np.sin(ylat*np.pi/180)
print(f.shape)

$$u_g = -\frac{g}{f} \frac{\partial z}{\partial y}$$
$$v_g = \frac{g}{f} \frac{\partial z}{\partial x}$$

$$u_a = u - u_g$$
$$v_a = v - v_g$$

In [ ]:
# calculate derivatives
dzdx=np.gradient(z,axis=3)/dx
dzdy=np.gradient(z,axis=2)/dy

# calculate geostrophic wind
ug=(-1)*(g/f)*dzdy
vg=(g/f)*dzdx

# calculate ageostrophic wind
ua = u - ug
va = v - vg

In [ ]:
ut=ug[:,lev500,:,:]-ug[:,lev925,:,:]
vt=vg[:,lev500,:,:]-vg[:,lev925,:,:]

# Diagnostics : divergence, tourbillon relatif

$$div(\vec{V}_h) = \frac{\partial u}{\partial x}+\frac{\partial v}{\partial y}-\frac{v}{a}tan(\phi)$$
$$div(\vec{V}_a) = \frac{\partial u_a}{\partial x}+\frac{\partial v_a}{\partial y}-\frac{v_a}{a}tan(\phi)$$

In [ ]:
ddx_u=np.gradient(u,axis=3)/dx
ddy_v=np.gradient(v,axis=2)/dy
div=ddx_u+ddy_v-(v/re)*np.tan(ylat*np.pi/180)

ddx_ua=np.gradient(ua,axis=3)/dx
ddy_va=np.gradient(va,axis=2)/dy
diva=ddx_ua+ddy_va-(va/re)*np.tan(ylat*np.pi/180)

print(div.shape)
print(np.min(div[:,lev300,:,:]))
print(np.max(div[:,lev300,:,:]))

$$\xi = \frac{\partial v}{\partial x}-\frac{\partial u}{\partial y}+\frac{u}{a}tan(\phi)$$
$$\xi_a = \xi+f$$

In [ ]:
ddx_v=np.gradient(v,axis=3)/dx
ddy_u=np.gradient(u,axis=2)/dy
vort=ddx_v-ddy_u+(u/re)*np.tan(ylat*np.pi/180)
absvort=vort+f

print(vort.shape)
print(absvort.shape)
print(np.min(vort[:,lev850,:,:]))
print(np.max(vort[:,lev850,:,:]))

# Diagnostics : tourbillon potentiel

$$PV = -g(\xi+f)\frac{\partial \theta}{\partial P}+g\left(\frac{\partial v}{\partial P}\frac{\partial \theta}{\partial x}-\frac{\partial u}{\partial P}\frac{\partial \theta}{\partial y}\right)$$

In [ ]:
# calculate derivatives
ddp_theta=np.gradient(theta,lev*100,axis=1)
ddx_theta=np.gradient(theta,axis=3)/dx
ddy_theta=np.gradient(theta,axis=2)/dy
ddp_u=np.gradient(u,lev*100,axis=1)
ddp_v=np.gradient(v,lev*100,axis=1)

# calculate contributions to PV and PV
pv_one=g*absvort*(-ddp_theta)
pv_two=g*(ddp_v*ddx_theta-ddp_u*ddy_theta)
pv2=pv_one+pv_two

print(pv2.shape)
print(np.min(pv2[:,lev300,:,:]))
print(np.max(pv2[:,lev300,:,:]))

# Diagnostics : advections de température et de tourbillon

In [ ]:
ddx_t=np.gradient(theta,axis=3)/dx
ddy_t=np.gradient(theta,axis=2)/dy
t_adv = (-1)*(u*ddx_t+v*ddy_t) *3*3600

In [ ]:
ddx_vort=np.gradient(absvort,axis=3)/dx
ddy_vort=np.gradient(absvort,axis=2)/dy
vort_adv = (-1)*(u*ddx_vort+v*ddy_vort)

# Paramètres pour les graphiques

In [ ]:
levels_msl = np.arange(900,1050,2)
levels_wind = [40, 50, 60, 70, 80, 90, 100, 110, 120]

levels_z925 = np.arange(300,2000,50)
levels_z850 = np.arange(1000,2100,50)
levels_z700 = np.arange(2000,4100,50)
levels_z500 = np.arange(4000,7100,50)
levels_z300 = np.arange(8000,10100,50)
levels_dz = np.arange(4200,5140,40)

levels_wpos = np.arange(0,4.2,0.2)
levels_wneg = np.arange(-4.2,0,0.2)

levels_rh = np.arange(90,100.5,0.5)
levels_the = np.arange(250,345,5)
levels_th = np.arange(int(theta[:,lev850,:,:].min()),int(theta[:,lev850,:,:].max()),1)

levels_vadv = np.arange(-20,22,2)
levels_tadv = np.arange(-10,11,1)

levels_vor = np.arange(5,95,5)
levels_div = np.arange(-15,16,1)

levels_ptend=np.arange(-26,26,2)

wind_slice = slice(None, None, 8)

https://unidata.github.io/python-gallery/examples/HILO_Symbol_Plot.html

In [ ]:
def plot_maxmin_points(data, extrema, nsize, symbol, color='k',
                       plotValue=True, transform=None):

    if (extrema == 'max'):
        data_ext = maximum_filter(data, nsize, mode='nearest')
    elif (extrema == 'min'):
        data_ext = minimum_filter(data, nsize, mode='nearest')
    else:
        raise ValueError('Value for hilo must be either max or min')

    mxy, mxx = np.where(data_ext == data)

    for i in range(len(mxy)):
        ax.text(data.longitude[mxx[i]].values, data.latitude[mxy[i]].values, symbol, color=color, size=12,
                clip_on=True, horizontalalignment='center', verticalalignment='center',
                transform=transform)
        ax.text(data.longitude[mxx[i]].values, data.latitude[mxy[i]].values,
                '\n' + str(int(data[mxy[i], mxx[i]])),
                color=color, size=10, clip_on=True, fontweight='bold',
                horizontalalignment='center', verticalalignment='top', transform=transform)

In [ ]:
def haversine(lon1, lat1, lon2, lat2):
    # convert decimal degrees to radians 
    lon1 = np.deg2rad(lon1)
    lon2 = np.deg2rad(lon2)
    lat1 = np.deg2rad(lat1)
    lat2 = np.deg2rad(lat2)

    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arcsin(np.sqrt(a)) 
    r = 6371
    return c * r

In [ ]:
projection = ccrs.PlateCarree()

coast = cfeature.NaturalEarthFeature(category='physical', scale='10m',
                            facecolor='none', name='coastline')
def plot_background(ax):
    ax.set_xticks(np.linspace(-180, 180, 37), crs=projection)
    ax.set_yticks(np.linspace(-90, 90, 19), crs=projection)
    lon_formatter = LongitudeFormatter(zero_direction_label=True)
    lat_formatter = LatitudeFormatter()
    ax.xaxis.set_major_formatter(lon_formatter)
    ax.yaxis.set_major_formatter(lat_formatter)
    ax.add_feature(coast, edgecolor='gray')
    return ax

In [ ]:
def make_animation():
    nbimages=len(time)
    # create a tuple of display durations, one for each frame
    first_last = 1000 #show the first and last frames for 100 ms
    standard_duration = 1000 #show all other frames for 5 ms
    durations = tuple([first_last] + [standard_duration] * (nbimages - 2) + [first_last])
    # load all the static images into a list
    images = [Image.open(image) for image in sorted(glob.glob('{}/*.png'.format('./anim/')))]
    # save as an animated gif
    gif = images[0]
    gif.info['duration'] = durations #ms per frame
    gif.info['loop'] = 0 #how many times to loop (0=infinite)
    gif.save(fp=gif_filepath, format='gif', save_all=True, append_images=images[1:])
    # verify that the number of frames in the gif equals the number of image files and durations
    Image.open(gif_filepath).n_frames == len(images) == len(durations)
    # clean png
    os.chdir("./anim/")
    for f in glob.glob("*.png"):
        os.remove(f)
    os.chdir("../")
    return Image

def make_animation2():
    nbimages=len(time)-1
    # create a tuple of display durations, one for each frame
    first_last = 1000 #show the first and last frames for 100 ms
    standard_duration = 1000 #show all other frames for 5 ms
    durations = tuple([first_last] + [standard_duration] * (nbimages - 2) + [first_last])
    # load all the static images into a list
    images = [Image.open(image) for image in sorted(glob.glob('{}/*.png'.format('./anim/')))]
    # save as an animated gif
    gif = images[0]
    gif.info['duration'] = durations #ms per frame
    gif.info['loop'] = 0 #how many times to loop (0=infinite)
    gif.save(fp=gif_filepath, format='gif', save_all=True, append_images=images[1:])
    # verify that the number of frames in the gif equals the number of image files and durations
    Image.open(gif_filepath).n_frames == len(images) == len(durations)
    # clean png
    os.chdir("./anim/")
    for f in glob.glob("*.png"):
        os.remove(f)
    os.chdir("../")
    return Image

# Cartes de visualisation de la situation

In [ ]:
bounds = [(lonW, lonE, latS, latN)]

In [ ]:
for i in range(1,len(time)):
    #print(date_str[i])
    fig = plt.figure(figsize=(17., 10.))
    ax = fig.add_subplot(1, 1, 1, projection=projection)
    ax.set_title('Mean Sea Level Pressure (hPa) and pressure tendency (hPa/6h)', loc='left', fontsize=14)
    ax.set_title(date_str[i], loc='right', fontsize=14)
    plot_background(ax)
    ax.set_extent(*bounds, crs=ccrs.PlateCarree())
    
    c = ax.contour(lon, lat, msl[i,:,:], levels_msl, colors='black', linewidths=1, transform=ccrs.PlateCarree())
    cf = ax.contourf(lon, lat, msl[i,:,:]-msl[i-1,:,:], levels=levels_ptend, extend='both',
                     cmap='bwr', transform=ccrs.PlateCarree())
    cb = fig.colorbar(cf, orientation='horizontal', aspect=65, shrink=0.5, pad=0.05, extendrect='True')

    cb.set_label('hPa/6h')
    plot_maxmin_points(msl[i,:,:], 'max', 25, symbol='H', color='r',  transform=ccrs.PlateCarree())
    plot_maxmin_points(msl[i,:,:], 'min', 25, symbol='L', color='b', transform=ccrs.PlateCarree())
    ax.clabel(c,fmt='%4.1i', fontsize=10)
    h,_ = c.legend_elements()
    ax.legend([h[0]], ['MSLP'], loc='lower right')
    
    figname='./anim/ECMWF_MSLP_Ptend_'+date_str[i]
    plt.savefig(figname+'.png',bbox_inches='tight')
    
    plt.show()
    plt.close()

gif_filepath = './anim/ECMWF_MSLP_Ptend_'+date1+'-'+date2+'.gif'
make_animation2()
IPdisplay.Image(url=gif_filepath)

In [ ]:
projection = ccrs.PlateCarree()
bounds = [(lonW, lonE, latS, latN)]

for i in range(len(time)):
    #print(date_str[i])
    fig = plt.figure(figsize=(17., 10.))
    ax = fig.add_subplot(1, 1, 1, projection=projection)
    ax.set_title('Mean Sea Level Pressure (hPa) and wind module (m/s) at 300 hPa', loc='left', fontsize=14)
    ax.set_title(date_str[i], loc='right', fontsize=14)
    plot_background(ax)
    ax.set_extent(*bounds, crs=ccrs.PlateCarree())
    
    cf = ax.contourf(lon, lat, uv_mod[i,lev300,:,:], levels_wind, transform=ccrs.PlateCarree(),
                     cmap='Spectral_r', extend='max')
    c = ax.contour(lon, lat, msl[i,:,:], levels_msl, colors='black', linewidths=1, transform=ccrs.PlateCarree())
    cb = fig.colorbar(cf, orientation='horizontal', aspect=65, shrink=0.5, pad=0.05, extendrect='True')
    cb.set_label('m/s')
    plot_maxmin_points(msl[i,:,:], 'max', 25, symbol='H', color='r',  transform=ccrs.PlateCarree())
    plot_maxmin_points(msl[i,:,:], 'min', 25, symbol='L', color='b', transform=ccrs.PlateCarree())
    ax.clabel(c,fmt='%4.1i', fontsize=10)
    h,_ = c.legend_elements()
    ax.legend([h[0]], ['MSLP'], loc='lower right')
    
    figname='./anim/ECMWF_MSLP_VMOD300_'+date_str[i]
    plt.savefig(figname+'.png',bbox_inches='tight')
    
    plt.show()
    plt.close()

gif_filepath = './anim/ECMWF_MSLP_VMOD300_'+date1+'-'+date2+'.gif'
make_animation()
IPdisplay.Image(url=gif_filepath)

In [ ]:
projection = ccrs.PlateCarree()
bounds = [(lonW, lonE, latS, latN)]

for i in range(len(time)):
    #print(date_str[i])
    fig = plt.figure(figsize=(17., 10.))
    ax = fig.add_subplot(1, 1, 1, projection=projection)
    ax.set_title('Geopotential height (mgp) and winds (kt) at 925 hPa, equivalent potential temperature (K) at 850 hPa',
                 loc='left', fontsize=14)
    ax.set_title(date_str[i], loc='right', fontsize=14)
    plot_background(ax)
    ax.set_extent(*bounds, crs=ccrs.PlateCarree())
    
    cf = ax.contourf(lon, lat, thetae[i,lev850,:,:], levels_the, transform=ccrs.PlateCarree(),
                     cmap='jet', extend='both')
    cb = fig.colorbar(cf, orientation='horizontal', aspect=65, shrink=0.5, pad=0.05, extendrect='True')
    cb.set_label('K')
    c = ax.contour(lon, lat, z[i,lev925,:,:], levels_z925, colors='black', linewidths=1, transform=ccrs.PlateCarree())
    ax.clabel(c,fmt='%4.1i', fontsize=10)
    h,_ = c.legend_elements()
    ax.legend([h[0]], ['Z925hPa'], loc='lower right')
    plot_maxmin_points(z[i,lev925,:,:], 'max', 25, symbol='H', color='r',  transform=ccrs.PlateCarree())
    plot_maxmin_points(z[i,lev925,:,:], 'min', 25, symbol='L', color='b', transform=ccrs.PlateCarree())
    
    ax.barbs(lon[wind_slice], lat[wind_slice],
         u[i,lev925][wind_slice, wind_slice],
         v[i,lev925][wind_slice, wind_slice], pivot='middle',
         color='red', length=6.5, transform=ccrs.PlateCarree())
    
    figname='./anim/ECMWF_ZUV925_Thetae850_'+date_str[i]
    plt.savefig(figname+'.png',bbox_inches='tight')
    
    plt.show()
    plt.close()
    
gif_filepath = './anim/ECMWF_ZUV925_Thetae850_'+date1+'-'+date2+'.gif'
make_animation()
IPdisplay.Image(url=gif_filepath)

In [ ]:
projection = ccrs.PlateCarree()
bounds = [(lonW, lonE, latS, latN)]

for i in range(len(time)):
    #print(date_str[i])
    fig = plt.figure(figsize=(17., 10.))
    ax = fig.add_subplot(1, 1, 1, projection=projection)
    ax.set_title('Equivalent potential temperature (K) at 850 hPa, geopotential (mgp) and cyclonic relative vorticity (10$^{-5}$ s$^{-1}$) at 925 hPa',
                 loc='left', fontsize=14)
    ax.set_title(date_str[i], loc='right', fontsize=14)
    plot_background(ax)
    ax.set_extent(*bounds, crs=ccrs.PlateCarree())
    
    cf = ax.contourf(lon, lat, thetae[i,lev850,:,:], levels_the, transform=ccrs.PlateCarree(),
                     cmap='jet', extend='both')
    cb = fig.colorbar(cf, orientation='horizontal', aspect=65, shrink=0.5, pad=0.05, extendrect='True')
    cb.set_label('K')
    
    c1 = ax.contour(lon, lat, z[i,lev925,:,:], levels_z925, colors='black', linewidths=1, transform=ccrs.PlateCarree())
    c2 = ax.contour(lon, lat, vort[i,lev925,:,:]*10**5, levels_vor, colors='green', transform=ccrs.PlateCarree(),
                    linewidths=1)                  
    ax.clabel(c1, fmt='%4.1i', fontsize=10)
    ax.clabel(c2, fmt='%4.1i', fontsize=10)
    
    h1,_ = c1.legend_elements()
    h2,_ = c2.legend_elements()    
    ax.legend([h1[0], h2[0]], ['Z925', 'vort925'], loc='lower right')
    
    figname='./anim/ECMWF_Zvort925_Thetae850_'+date_str[i]
    plt.savefig(figname+'.png',bbox_inches='tight')
    
    plt.show()
    plt.close()

gif_filepath = './anim/ECMWF_Zvort925_Thetae850_'+date1+'-'+date2+'.gif'
make_animation()
IPdisplay.Image(url=gif_filepath)

# Vent thermique et advection thermique

In [ ]:
date_selection="2014-01-26T00"
itime = date_str.index(date_selection)
bounds = [(-40, 0, 40, 65)]

In [ ]:
fig = plt.figure(figsize=(17., 10.))
ax = fig.add_subplot(1, 1, 1, projection=projection)
ax.set_title('Geopotential heights ', fontsize=14)
ax.set_title(date_selection, loc='right', fontsize=14)
plot_background(ax)
ax.set_extent(*bounds, crs=ccrs.PlateCarree())
   
c1 = ax.contour(lon, lat, z[itime,lev925,:,:], levels_z925, colors='black', linewidths=1, transform=ccrs.PlateCarree())
c2 = ax.contour(lon, lat, z[itime,lev500,:,:], levels_z500, colors='blue', linewidths=1, transform=ccrs.PlateCarree())
ax.clabel(c1,fmt='%4.1i', fontsize=10)
ax.clabel(c2,fmt='%4.1i', fontsize=10)

h1,_ = c1.legend_elements()
h2,_ = c2.legend_elements()
ax.legend([h1[0], h2[0]], ['Z925', 'Z500'], loc='lower right')

figname='./figs/ECMWF_thermal_wind0_'+date_selection
plt.savefig(figname+'.png')

plt.show()

In [ ]:
fig = plt.figure(figsize=(17., 10.))
ax = fig.add_subplot(1, 1, 1, projection=projection)
ax.set_title('Geopotential heights and geostrophic winds', fontsize=14)
ax.set_title(date_selection, loc='right', fontsize=14)
plot_background(ax)
ax.set_extent(*bounds, crs=ccrs.PlateCarree())
   
c1 = ax.contour(lon, lat, z[itime,lev925,:,:], levels_z925, colors='black', linewidths=1, transform=ccrs.PlateCarree())
c2 = ax.contour(lon, lat, z[itime,lev500,:,:], levels_z500, colors='blue', linewidths=1, transform=ccrs.PlateCarree())
ax.clabel(c1,fmt='%4.1i', fontsize=10)
ax.clabel(c2,fmt='%4.1i', fontsize=10)

ax.barbs(lon[wind_slice], lat[wind_slice],
         ug[itime,lev925,:,:][wind_slice, wind_slice],
         vg[itime,lev925,:,:][wind_slice, wind_slice], pivot='middle',
         color='k', length=6.5, transform=ccrs.PlateCarree())

ax.barbs(lon[wind_slice], lat[wind_slice],
         ug[itime,lev500,:,:][wind_slice, wind_slice],
         vg[itime,lev500,:,:][wind_slice, wind_slice], pivot='middle',
         color='blue', length=6.5, transform=ccrs.PlateCarree())

h1,_ = c1.legend_elements()
h2,_ = c2.legend_elements()
ax.legend([h1[0], h2[0]], ['Z925', 'Z500'], loc='lower right')

figname='./figs/ECMWF_thermal_wind1_'+date_selection
plt.savefig(figname+'.png')

plt.show()

In [ ]:
fig = plt.figure(figsize=(15., 12.))
ax = fig.add_subplot(1, 1, 1, projection=projection)
ax.set_title('Geopotential heights and thermal wind', fontsize=14)
ax.set_title(date_selection, loc='right', fontsize=14)
plot_background(ax)
ax.set_extent(*bounds, crs=ccrs.PlateCarree())
   
c1 = ax.contour(lon, lat, z[itime,lev925,:,:], levels_z925, colors='black', linewidths=1, transform=ccrs.PlateCarree())
c2 = ax.contour(lon, lat, z[itime,lev500,:,:], levels_z500, colors='blue', linewidths=1, transform=ccrs.PlateCarree())
c3 = ax.contour(lon, lat, dz[itime,:,:], levels_dz, colors='red', linewidths=1, transform=ccrs.PlateCarree())
ax.clabel(c1,fmt='%4.1i', fontsize=10)
ax.clabel(c2,fmt='%4.1i', fontsize=10)
ax.clabel(c3,fmt='%4.1i', fontsize=10)

ax.barbs(lon[wind_slice], lat[wind_slice],
         ut[itime,:,:][wind_slice, wind_slice],
         vt[itime,:,:][wind_slice, wind_slice], pivot='middle',
         color='red', length=6.5, transform=ccrs.PlateCarree())

h1,_ = c1.legend_elements()
h2,_ = c2.legend_elements()
h3,_ = c3.legend_elements()
ax.legend([h1[0], h2[0], h3[0]], ['Z925', 'Z500', 'Z500 - Z925'],
          loc='lower right')

figname='./figs/ECMWF_thermal_wind2_'+date_selection
plt.savefig(figname+'.png')

plt.show()

In [ ]:
fig = plt.figure(figsize=(15., 12.))
ax = fig.add_subplot(1, 1, 1, projection=projection)
ax.set_title('Geopotential height, potential temperature and horizontal wind at 850 hPa', fontsize=14)
ax.set_title(date_selection, loc='right', fontsize=14)
plot_background(ax)
ax.set_extent(*bounds, crs=ccrs.PlateCarree())

cf = ax.contourf(lon, lat, theta[itime,lev850,:,:], levels_th, transform=ccrs.PlateCarree(), cmap='jet', extend='both')
cb = fig.colorbar(cf, orientation='horizontal', aspect=65, shrink=0.5, pad=0.05, extendrect='True')
cb.set_label('Potential temperature (K)')
 
c2 = ax.contour(lon, lat, theta[itime,lev850,:,:], levels_th, transform=ccrs.PlateCarree(),
                colors='blue', linewidths=1, linestyles='--')                 
c3 = ax.contour(lon, lat, z[itime,lev850,:,:], levels_z850, colors='black', linewidths=1, transform=ccrs.PlateCarree())
ax.clabel(c3,fmt='%4.1i', fontsize=10)

h2,_ = c2.legend_elements()
h3,_ = c3.legend_elements()
ax.legend([h2[0], h3[0]], [r'$\theta$850', 'Z850'], loc='lower right')

ax.barbs(lon[wind_slice], lat[wind_slice],
         u[itime,lev850,:,:][wind_slice, wind_slice],
         v[itime,lev850,:,:][wind_slice, wind_slice], pivot='middle',
         color='red', length=6.5, transform=ccrs.PlateCarree())

figname='./figs/ECMWF_thermal_wind3_'+date_selection
plt.savefig(figname+'.png')

plt.show()

In [ ]:
fig = plt.figure(figsize=(15., 12.))
ax = fig.add_subplot(1, 1, 1, projection=projection)
ax.set_title('Geopotential heigh, potential temperature and temperature advection at 850 hPa', fontsize=14)
ax.set_title(date_selection, loc='right', fontsize=14)
plot_background(ax)
ax.set_extent(*bounds, crs=ccrs.PlateCarree())

cf = ax.contourf(lon, lat, t_adv[itime,lev850,:,:], levels_tadv[levels_tadv != 0], extend='both',
                 cmap='bwr', transform=ccrs.PlateCarree())
cb = fig.colorbar(cf, orientation='horizontal', aspect=65, shrink=0.5, pad=0.05, extendrect='True')
cb.set_label('Temperature advection (K/3h)')

c2 = ax.contour(lon, lat, theta[itime,lev850,:,:], levels_th, transform=ccrs.PlateCarree(),
                colors='blue', linewidths=1, linestyles='--')                  
c3 = ax.contour(lon, lat, z[itime,lev850,:,:], levels_z850, colors='black', linewidths=1, transform=ccrs.PlateCarree())
ax.clabel(c2,fmt='%4.1i', fontsize=10)
ax.clabel(c3,fmt='%4.1i', fontsize=10)

h2,_ = c2.legend_elements()
h3,_ = c3.legend_elements()
ax.legend([h2[0], h3[0]], [r'$\theta$850', 'Z850'], loc='lower right')

figname='./figs/ECMWF_thermal_wind4_'+date_selection
plt.savefig(figname+'.png')

plt.show()

# Tourbillon / Circulation

In [ ]:
fig = plt.figure(figsize=(15., 12.))
ax = fig.add_subplot(1, 1, 1, projection=projection)
ax.set_title('Geopotential height and relative vorticity at 925 hPa', loc='right', fontsize=14)
ax.set_title(date_selection, loc='right', fontsize=14)
plot_background(ax)
ax.set_extent(*bounds, crs=ccrs.PlateCarree())
   
c1 = ax.contour(lon, lat, z[itime,lev925,:,:], levels_z925, colors='black', linewidths=1, transform=ccrs.PlateCarree())
cf=ax.contourf(lon, lat, vort[itime,lev925,:,:]*10**5, levels_vor, transform=ccrs.PlateCarree(), cmap='Spectral_r', extend='max')
ax.clabel(c1,fmt='%4.1i', fontsize=10)
cb = fig.colorbar(cf, orientation='horizontal', aspect=65, shrink=0.5, pad=0.05, extendrect='True')
cb.set_label('Relative vortiticy $(10^5 s^{-1}$)')

h1,_ = c1.legend_elements()
ax.legend([h1[0]], ['Z925'], loc='lower right')

figname='./figs/ECMWF_vort925_'+date_str[itime]
plt.savefig(figname+'.png')

plt.show()

In [ ]:
max_dist=500
# Center coordinates
clat = 55.5
clon = -23
# Calculate distance between center and all other lat/lon pairs
distance = haversine(lon[:,np.newaxis], lat, clon, clat)
distance2=np.transpose(distance)
# Mask distance array where distance > max_dist
distance_m = np.ma.masked_greater(distance2, max_dist)
# Mask the data array based on the distance mask
vort_m = np.ma.masked_where(distance2 > max_dist, vort[itime,lev925,:,:])
z_m = np.ma.masked_where(distance2 > max_dist, z[itime,lev925,:,:])

vort_mean=np.nanmean(vort_m)
circulation=vort_mean*np.pi*(max_dist*1000)**2

In [ ]:
print('Tourbillon relatif moyen dans la zone choisie (s⁻¹) : ', np.array("%e"%vort_mean))
print('Circulation associée (m²s⁻¹,) : ', "%e"%circulation)

fig = plt.figure(figsize=(15., 12.))
ax = fig.add_subplot(1, 1, 1, projection=projection)
ax.set_title('Geopotential height and relative vorticity at 925 hPa', loc='right', fontsize=14)
ax.set_title(date_selection, loc='right', fontsize=14)
plot_background(ax)
ax.gridlines()
ax.set_extent(*bounds, crs=ccrs.PlateCarree())
#ax.scatter(clon,clat, color='g', transform=ccrs.PlateCarree())
         
c1 = ax.contour(lon, lat, z_m, levels_z925, colors='black', linewidths=1, transform=ccrs.PlateCarree())
cf=ax.contourf(lon, lat, vort_m*10**5, levels_vor, transform=ccrs.PlateCarree(), cmap='Spectral_r', extend='max')
ax.clabel(c1,fmt='%4.1i', fontsize=10)
cb = fig.colorbar(cf, orientation='horizontal', aspect=65, shrink=0.5, pad=0.05, extendrect='True')
cb.set_label('Relative vortiticy $(10^5 s^{-1}$)')

h1,_ = c1.legend_elements()
ax.legend([h1[0]], ['Z925'], loc='lower right')

figname='./figs/ECMWF_vort925_zoom_'+date_str[itime]
plt.savefig(figname+'.png')

plt.show()

# Vent agéostrophique et vitesses verticales

In [ ]:
date_selection="2014-01-26T00"
itime = date_str.index(date_selection)
bounds = [(-40, 0, 40, 65)]

In [ ]:
fig = plt.figure(figsize=(15., 12.))
ax = fig.add_subplot(1, 1, 1, projection=projection)
ax.set_title('Geopotential height (mgp) and and horizontal wind at 925 hPa, MSLP tendency (hPa/6h)', loc='left', fontsize=14)
ax.set_title(date_str[itime], loc='right', fontsize=14)
plot_background(ax)
ax.set_extent(*bounds, crs=ccrs.PlateCarree())
    
c1 = ax.contour(lon, lat, z[itime,lev925,:,:], levels_z925, colors='black', linewidths=1,
                transform=ccrs.PlateCarree())
ax.clabel(c1,fmt='%4.1i', fontsize=10)

cf = ax.contourf(lon, lat, msl[itime,:,:]-msl[itime-1,:,:], levels=levels_ptend, extend='both',
                 cmap='bwr', transform=ccrs.PlateCarree())
cb = fig.colorbar(cf, orientation='horizontal', aspect=65, shrink=0.5, pad=0.05, extendrect='True')
cb.set_label('hPa/6h')

ax.barbs(lon[wind_slice], lat[wind_slice],
         u[itime,lev925][wind_slice, wind_slice],
         v[itime,lev925][wind_slice, wind_slice], pivot='middle',
         color='green', length=6.5, transform=ccrs.PlateCarree())

plot_maxmin_points(msl[itime,:,:], 'max', 25, symbol='H', color='r',  transform=ccrs.PlateCarree())
plot_maxmin_points(msl[itime,:,:], 'min', 25, symbol='L', color='b', transform=ccrs.PlateCarree())

h,_ = c1.legend_elements()
ax.legend([h[0]], ['Z925'], loc='lower right')
    
figname='./figs/ECMWF_ZVh925_Ptend_'+date_str[itime]
plt.savefig(figname+'.png',bbox_inches='tight')
    
plt.show()

In [ ]:
fig = plt.figure(figsize=(15., 12.))
ax = fig.add_subplot(1, 1, 1, projection=projection)
ax.set_title('Geopotential height (mgp) and and ageostrophic wind at 925 hPa, MSLP tendency (hPa/6h)', loc='left', fontsize=14)
ax.set_title(date_str[itime], loc='right', fontsize=14)
plot_background(ax)
ax.set_extent(*bounds, crs=ccrs.PlateCarree())
    
c1 = ax.contour(lon, lat, z[itime,lev925,:,:], levels_z925, colors='black', linewidths=1,
                transform=ccrs.PlateCarree())
ax.clabel(c1,fmt='%4.1i', fontsize=10)

cf = ax.contourf(lon, lat, msl[itime,:,:]-msl[itime-1,:,:], levels=levels_ptend, extend='both',
                 cmap='bwr', transform=ccrs.PlateCarree())
cb = fig.colorbar(cf, orientation='horizontal', aspect=65, shrink=0.5, pad=0.05, extendrect='True')
cb.set_label('hPa/6h')

ax.barbs(lon[wind_slice], lat[wind_slice],
         ua[itime,lev925][wind_slice, wind_slice],
         va[itime,lev925][wind_slice, wind_slice], pivot='middle',
         color='green', length=6.5, transform=ccrs.PlateCarree())

plot_maxmin_points(msl[itime,:,:], 'max', 25, symbol='H', color='r',  transform=ccrs.PlateCarree())
plot_maxmin_points(msl[itime,:,:], 'min', 25, symbol='L', color='b', transform=ccrs.PlateCarree())
h,_ = c1.legend_elements()
ax.legend([h[0]], ['Z925'], loc='lower right')
    
figname='./figs/ECMWF_ZVa925_Ptend_'+date_str[itime]
plt.savefig(figname+'.png',bbox_inches='tight')
    
plt.show()

In [ ]:
div925=gaussian_filter(diva[itime,lev925,:,:]*1e05, sigma=2.0)
#div925=diva[itime,lev925,:,:]*1e05

fig = plt.figure(figsize=(15., 12.))
ax = fig.add_subplot(1, 1, 1, projection=projection)
ax.set_title('Geopotential height (mgp) and and ageostrophic divergence ($10^{-5}$s$^{-1}$) at 925 hPa, MSLP tendency (hPa/6h)', loc='left', fontsize=14)
ax.set_title(date_str[itime], loc='right', fontsize=14)
plot_background(ax)
ax.set_extent(*bounds, crs=ccrs.PlateCarree())
    
c1 = ax.contour(lon, lat, z[itime,lev925,:,:], levels_z925, colors='black', linewidths=1,
                transform=ccrs.PlateCarree())
ax.clabel(c1,fmt='%4.1i', fontsize=10)
c2 = ax.contour(lon, lat, div925, levels_div[levels_div != 0], colors='green', linewidths=1, transform=ccrs.PlateCarree())
ax.clabel(c2,fmt='%4.1i', fontsize=10)

cf = ax.contourf(lon, lat, msl[itime,:,:]-msl[itime-1,:,:], levels=levels_ptend, extend='both',
                 cmap='bwr', transform=ccrs.PlateCarree())
cb = fig.colorbar(cf, orientation='horizontal', aspect=65, shrink=0.5, pad=0.05, extendrect='True')
cb.set_label('hPa/6h')

plot_maxmin_points(msl[itime,:,:], 'max', 25, symbol='H', color='r',  transform=ccrs.PlateCarree())
plot_maxmin_points(msl[itime,:,:], 'min', 25, symbol='L', color='b', transform=ccrs.PlateCarree())

h1,_ = c1.legend_elements()
h2,_ = c2.legend_elements()
ax.legend([h1[0], h2[0]], ['Z925hPa', 'div925'], loc='lower right')
    
figname='./figs/ECMWF_ZdivVa925_Ptend_'+date_str[itime]
plt.savefig(figname+'.png',bbox_inches='tight')
    
plt.show()

In [ ]:
div925=gaussian_filter(diva[itime,lev925,:,:]*1e05, sigma=2.0)
#div925=diva[itime,lev925,:,:]*1e05

fig = plt.figure(figsize=(15., 12.))
ax = fig.add_subplot(1, 1, 1, projection=projection)
ax.set_title('Geopotential height (mgp) and ageostrophic divergence ($10^{-5}$s$^{-1}$) at 925 hPa - Vertical motion (Pa/s) at 600 hPa',
             loc='left', fontsize=14)
ax.set_title(date_str[itime], loc='right', fontsize=14)
plot_background(ax)
ax.set_extent(*bounds, crs=ccrs.PlateCarree())

cf = ax.contourf(lon, lat, div925,
                 levels_div[levels_div != 0], extend='both',
                 cmap='bwr', transform=ccrs.PlateCarree())
cb = fig.colorbar(cf, orientation='horizontal', aspect=65, shrink=0.5, pad=0.05, extendrect='True')

cb.set_label('$10^{-5}$s$^{-1}$')

c1 = ax.contour(lon, lat, z[itime,lev925,:,:], levels_z925, colors='black', linewidths=1,
                transform=ccrs.PlateCarree())
c2 = ax.contour(lon, lat, w[itime,lev600,:,:], levels_wpos[levels_wpos != 0], colors='brown', linewidths=1,
                transform=ccrs.PlateCarree()) 
c3 = ax.contour(lon, lat, w[itime,lev600,:,:], levels_wneg[levels_wneg != 0], colors='green', linewidths=1,
                linestyle="--", transform=ccrs.PlateCarree()) 

ax.clabel(c1, fmt='%4.1i', fontsize=10)
ax.clabel(c2, fmt='%4.1f', fontsize=10)
ax.clabel(c3, fmt='%4.1f', fontsize=10)

h1,_ = c1.legend_elements()
h2,_ = c2.legend_elements()
h3,_ = c3.legend_elements()
ax.legend([h1[0], h2[0], h3[0]], ['Z925hPa', 'VV+ 600hPa', 'VV- 600hPa'], loc='lower right')

figname='./figs/ECMWF_Zdiv925_VV600_'+date_str[itime]
plt.savefig(figname+'.png',bbox_inches='tight')

plt.show()

In [ ]:
levels_rh=np.arange(80,102,2)

rh700=rh[itime,lev700,:,:]*100
rh300=rh[itime,lev300,:,:]*100

fig = plt.figure(figsize=(15., 12.))
ax = fig.add_subplot(1, 1, 1, projection=projection)
ax.set_title('Geopotential height (mgp) at 925 hPa, RH>80% at 700 hPa - Vertical motion (Pa/s) at 600 hPa',
             loc='left', fontsize=14)
ax.set_title(date_str[itime], loc='right', fontsize=14)
plot_background(ax)
ax.set_extent(*bounds, crs=ccrs.PlateCarree())

cf = ax.contourf(lon, lat, rh700,
                 levels_rh, extend='max',
                 cmap='magma', transform=ccrs.PlateCarree())
cb = fig.colorbar(cf, orientation='horizontal', aspect=65, shrink=0.5, pad=0.05, extendrect='True')
cb.set_label('%')

c1 = ax.contour(lon, lat, z[itime,lev925,:,:], levels_z925, colors='black', linewidths=1,
                transform=ccrs.PlateCarree())
c2 = ax.contour(lon, lat, w[itime,lev600,:,:], levels_wpos[levels_wpos != 0], colors='brown', linewidths=1,
                transform=ccrs.PlateCarree()) 
c3 = ax.contour(lon, lat, w[itime,lev600,:,:], levels_wneg[levels_wneg != 0], colors='green', linewidths=1,
                linestyle="--", transform=ccrs.PlateCarree()) 

ax.clabel(c1, fmt='%4.1i', fontsize=10)
ax.clabel(c2, fmt='%4.1f', fontsize=10)
ax.clabel(c3, fmt='%4.1f', fontsize=10)

h1,_ = c1.legend_elements()
h2,_ = c2.legend_elements()
h3,_ = c3.legend_elements()
ax.legend([h1[0], h2[0], h3[0]], ['Z925hPa', 'VV+ 600hPa', 'VV- 600hPa'], loc='lower right')

figname='./figs/ECMWF_Z925_RH700_VV600_'+date_str[itime]
plt.savefig(figname+'.png',bbox_inches='tight')

plt.show()

# Advection thermique et vitesses verticales

In [ ]:
tadv850=gaussian_filter(t_adv[itime,lev850,:,:], sigma=3.0)
#tadv850=t_adv[itime,lev850,:,:]

fig = plt.figure(figsize=(15., 12.))
ax = fig.add_subplot(1, 1, 1, projection=projection)
ax.set_title('Geopotential height (mgp) and temperature advection (K/3h) at 850 hPa - Vertical motion (Pa/s) at 600 hPa',
             loc='left', fontsize=14)
ax.set_title(date_str[itime], loc='right', fontsize=14)
plot_background(ax)
ax.set_extent(*bounds, crs=ccrs.PlateCarree())

cf = ax.contourf(lon, lat, tadv850,
                 levels_tadv[levels_tadv != 0], extend='both',
                 cmap='bwr', transform=ccrs.PlateCarree())
cb = fig.colorbar(cf, orientation='horizontal', aspect=65, shrink=0.5, pad=0.05, extendrect='True')
cb.set_label('K/3h')

c1 = ax.contour(lon, lat, z[itime,lev850,:,:], levels_z850, colors='black', linewidths=1,
                transform=ccrs.PlateCarree())
c2 = ax.contour(lon, lat, w[itime,lev600,:,:], levels_wpos[levels_wpos != 0], colors='brown', linewidths=1,
                transform=ccrs.PlateCarree()) 
c3 = ax.contour(lon, lat, w[itime,lev600,:,:], levels_wneg[levels_wneg != 0], colors='green', linewidths=1,
                linestyle="--", transform=ccrs.PlateCarree()) 

ax.clabel(c1, fmt='%4.1i', fontsize=10)
ax.clabel(c2, fmt='%4.1f', fontsize=10)
ax.clabel(c3, fmt='%4.1f', fontsize=10)

h1,_ = c1.legend_elements()
h2,_ = c2.legend_elements()
h3,_ = c3.legend_elements()
ax.legend([h1[0], h2[0], h3[0]], ['Z850hPa', 'VV+ 600hPa', 'VV- 600hPa'], loc='lower right')

figname='./figs/ECMWF_ZTadv850_VV600_'+date_str[itime]
plt.savefig(figname+'.png',bbox_inches='tight')

plt.show()

# Advection différentielle de tourbillon et vitesses verticales

In [ ]:
vort_adv300=gaussian_filter(vort_adv[itime,lev300,:,:], sigma=5.0)
vort_adv850=gaussian_filter(vort_adv[itime,lev850,:,:], sigma=5.0)

#vort_adv300=vort_adv[itime,lev300,:,:]
#vort_adv850=vort_adv[itime,lev850,:,:]

fig = plt.figure(figsize=(15., 12.))
ax = fig.add_subplot(1, 1, 1, projection=projection)
ax.set_title('300hPa geopotential height (mgp), differential vorticity advection ($10^{-9}$s$^{-2}$) 600hPa vertical motion (Pa/s)',
             loc='left', fontsize=14)
ax.set_title(date_str[itime], loc='right', fontsize=14)
plot_background(ax)
ax.set_extent(*bounds, crs=ccrs.PlateCarree())

cf = ax.contourf(lon, lat,
                 (vort_adv300-vort_adv850)* 1e9,
                 levels_vadv[levels_vadv != 0],
                 extend='both', cmap='bwr', transform=ccrs.PlateCarree())
cb = fig.colorbar(cf, orientation='horizontal', aspect=65, shrink=0.5, pad=0.05, extendrect='True')
cb.set_label('$10^{-9}$s$^{-2}$')

c1 = ax.contour(lon, lat, z[itime,lev300,:,:], levels_z300, colors='black', linewidths=1, transform=ccrs.PlateCarree())
c2 = ax.contour(lon, lat, w[itime,lev600,:,:], levels_wpos[levels_wpos != 0], colors='brown', linewidths=1,
                transform=ccrs.PlateCarree()) 
c3 = ax.contour(lon, lat, w[itime,lev600,:,:], levels_wneg[levels_wneg != 0], colors='green', linewidths=1,
                linestyle="--", transform=ccrs.PlateCarree()) 

ax.clabel(c1, fmt='%4.1i', fontsize=10)
ax.clabel(c2, fmt='%4.1f', fontsize=10)
ax.clabel(c3, fmt='%4.1f', fontsize=10)

h1,_ = c1.legend_elements()
h2,_ = c2.legend_elements()
h3,_ = c3.legend_elements()
ax.legend([h1[0], h2[0], h3[0]], ['Z300hPa', 'VV+ 600hPa', 'VV- 600hPa'], loc='lower right')

figname='./figs/ECMWF_Zvortadv300_850_VV600_'+date_str[itime]
plt.savefig(figname+'.png',bbox_inches='tight')

plt.show()

# Forçages de vitesses verticales

In [ ]:
id=np.all([tadv850>0,(vort_adv300-vort_adv850)>0],axis=0)

fig = plt.figure(figsize=(15., 12.))
ax = fig.add_subplot(1, 1, 1, projection=projection)
ax.set_title('925hPa geopotential height (mgp), 600hPa vertical motion (Pa/s) : Tadv+ VORTadv+',
             loc='left', fontsize=14)
ax.set_title(date_str[itime], loc='right', fontsize=14)
plot_background(ax)
ax.set_extent(*bounds, crs=ccrs.PlateCarree())

ax.scatter(xlon[id], ylat[id], color='cyan')

c1 = ax.contour(lon, lat, z[itime,lev925,:,:], levels_z925, colors='black', linewidths=1, transform=ccrs.PlateCarree())
c2 = ax.contour(lon, lat, w[itime,lev600,:,:], levels_wpos[levels_wpos != 0], colors='brown', linewidths=1,
                transform=ccrs.PlateCarree()) 
c3 = ax.contour(lon, lat, w[itime,lev600,:,:], levels_wneg[levels_wneg != 0], colors='green', linewidths=1,
                linestyle="--", transform=ccrs.PlateCarree()) 

ax.clabel(c1, fmt='%4.1i', fontsize=10)
ax.clabel(c2, fmt='%4.1f', fontsize=10)
ax.clabel(c3, fmt='%4.1f', fontsize=10)

h1,_ = c1.legend_elements()
h2,_ = c2.legend_elements()
h3,_ = c3.legend_elements()
ax.legend([h1[0], h2[0], h3[0]], ['Z925hPa', 'VV+ 600hPa', 'VV- 600hPa'], loc='lower right')

figname='./figs/ECMWF_VV_forcing1_'+date_str[itime]
plt.savefig(figname+'.png',bbox_inches='tight')

plt.show()

In [ ]:
id=np.all([tadv850<0,(vort_adv300-vort_adv850)<0],axis=0)

fig = plt.figure(figsize=(15., 12.))
ax = fig.add_subplot(1, 1, 1, projection=projection)
ax.set_title('925hPa geopotential height (mgp), 600hPa vertical motion (Pa/s) : Tadv- VORTadv-',
             loc='left', fontsize=14)
ax.set_title(date_str[itime], loc='right', fontsize=14)
plot_background(ax)
ax.set_extent(*bounds, crs=ccrs.PlateCarree())

ax.scatter(xlon[id], ylat[id], color='orange')

c1 = ax.contour(lon, lat, z[itime,lev925,:,:], levels_z925, colors='black', linewidths=1, transform=ccrs.PlateCarree())
c2 = ax.contour(lon, lat, w[itime,lev600,:,:], levels_wpos[levels_wpos != 0], colors='brown', linewidths=1,
                transform=ccrs.PlateCarree()) 
c3 = ax.contour(lon, lat, w[itime,lev600,:,:], levels_wneg[levels_wneg != 0], colors='green', linewidths=1,
                linestyle="--", transform=ccrs.PlateCarree()) 

ax.clabel(c1, fmt='%4.1i', fontsize=10)
ax.clabel(c2, fmt='%4.1f', fontsize=10)
ax.clabel(c3, fmt='%4.1f', fontsize=10)

h1,_ = c1.legend_elements()
h2,_ = c2.legend_elements()
h3,_ = c3.legend_elements()
ax.legend([h1[0], h2[0], h3[0]], ['Z925hPa', 'VV+ 600hPa', 'VV- 600hPa'], loc='lower right')

figname='./figs/ECMWF_VV_forcing2_'+date_str[itime]
plt.savefig(figname+'.png',bbox_inches='tight')

plt.show()

In [ ]:
id=np.all([tadv850>0,(vort_adv300-vort_adv850)<0],axis=0)

fig = plt.figure(figsize=(15., 12.))
ax = fig.add_subplot(1, 1, 1, projection=projection)
ax.set_title('925hPa geopotential height (mgp), 600hPa vertical motion (Pa/s) : Tadv+ VORTadv-',
             loc='left', fontsize=14)
ax.set_title(date_str[itime], loc='right', fontsize=14)
plot_background(ax)
ax.set_extent(*bounds, crs=ccrs.PlateCarree())

ax.scatter(xlon[id], ylat[id], color='pink')

c1 = ax.contour(lon, lat, z[itime,lev925,:,:], levels_z925, colors='black', linewidths=1, transform=ccrs.PlateCarree())
c2 = ax.contour(lon, lat, w[itime,lev600,:,:], levels_wpos[levels_wpos != 0], colors='brown', linewidths=1,
                transform=ccrs.PlateCarree()) 
c3 = ax.contour(lon, lat, w[itime,lev600,:,:], levels_wneg[levels_wneg != 0], colors='green', linewidths=1,
                linestyle="--", transform=ccrs.PlateCarree()) 

ax.clabel(c1, fmt='%4.1i', fontsize=10)
ax.clabel(c2, fmt='%4.1f', fontsize=10)
ax.clabel(c3, fmt='%4.1f', fontsize=10)

h1,_ = c1.legend_elements()
h2,_ = c2.legend_elements()
h3,_ = c3.legend_elements()
ax.legend([h1[0], h2[0], h3[0]], ['Z925hPa', 'VV+ 600hPa', 'VV- 600hPa'], loc='lower right')

figname='./figs/ECMWF_VV_forcing3_'+date_str[itime]
plt.savefig(figname+'.png',bbox_inches='tight')

plt.show()

In [ ]:
id=np.all([tadv850<0,(vort_adv300-vort_adv850)>0],axis=0)

fig = plt.figure(figsize=(15., 12.))
ax = fig.add_subplot(1, 1, 1, projection=projection)
ax.set_title('925hPa geopotential height (mgp), 600hPa vertical motion (Pa/s) : Tadv- VORTadv+',
             loc='left', fontsize=14)
ax.set_title(date_str[itime], loc='right', fontsize=14)
plot_background(ax)
ax.set_extent(*bounds, crs=ccrs.PlateCarree())

ax.scatter(xlon[id], ylat[id], color='yellow')

c1 = ax.contour(lon, lat, z[itime,lev925,:,:], levels_z925, colors='black', linewidths=1, transform=ccrs.PlateCarree())
c2 = ax.contour(lon, lat, w[itime,lev600,:,:], levels_wpos[levels_wpos != 0], colors='brown', linewidths=1,
                transform=ccrs.PlateCarree()) 
c3 = ax.contour(lon, lat, w[itime,lev600,:,:], levels_wneg[levels_wneg != 0], colors='green', linewidths=1,
                linestyle="--", transform=ccrs.PlateCarree()) 

ax.clabel(c1, fmt='%4.1i', fontsize=10)
ax.clabel(c2, fmt='%4.1f', fontsize=10)
ax.clabel(c3, fmt='%4.1f', fontsize=10)

h1,_ = c1.legend_elements()
h2,_ = c2.legend_elements()
h3,_ = c3.legend_elements()
ax.legend([h1[0], h2[0], h3[0]], ['Z925hPa', 'VV+ 600hPa', 'VV- 600hPa'], loc='lower right')

figname='./figs/ECMWF_VV_forcing4_'+date_str[itime]
plt.savefig(figname+'.png',bbox_inches='tight')

plt.show()

# Etude du champ de tourbillon potentiel

In [ ]:
levs_pv = np.arange(-1, 10.5, 0.5)
levs_pv2 = np.arange(0, 10.5, 0.5)
levs_pv3 = np.arange(5, 10.5, 0.5)
levs_2pvu = np.arange(2, 2.5, 0.5)
levs_15pvu = np.arange(1.5, 2, 0.5)
levs_th = np.arange(100, 500, 5)
levs_the = np.arange(150,550,5)

cmap='YlOrRd'
cmap='viridis'

In [ ]:
projection = ccrs.PlateCarree()
bounds = [(lonW, lonE, latS, latN)]

for i in range(len(time)):
    print(date_str[i])
    
    fig = plt.figure(figsize=(15., 15.))
    fig.suptitle('Potential vorticity at 300 hPa : '+date_str[i], fontsize=16)
    
    ax = plt.subplot(211, projection=projection)
    ax.set_title('from netcdf', loc='center')
    plot_background(ax)
    ax.set_extent(*bounds, ccrs.PlateCarree())
    cs1 = ax.contourf(lon, lat, pv[i,lev300,:,:]*1e6, levs_pv, cmap=cmap,
                      extend='both', transform=ccrs.PlateCarree())
    cb = plt.colorbar(cs1, orientation='horizontal', aspect=65, shrink=0.5, pad=0.05, extendrect='True')
    cb.set_label('10$^{-6}$ K  m$^{2}$ kg$^{-1}$ s$^{-1}$')
    
    ax = plt.subplot(212, projection=projection)
    ax.set_title('diagnostic', loc='center')
    plot_background(ax)
    ax.set_extent(*bounds, ccrs.PlateCarree())
    cs1 = ax.contourf(lon, lat, pv2[i,lev300,:,:]*1e6, levs_pv, cmap=cmap,
                      extend='both', transform=ccrs.PlateCarree())
    cb = plt.colorbar(cs1, orientation='horizontal', aspect=65, shrink=0.5, pad=0.05, extendrect='True')
    cb.set_label('10$^{-6}$ K  m$^{2}$ kg$^{-1}$ s$^{-1}$')
    
    figname='./anim/ERA5_PV_'+date_str[i]
    plt.savefig(figname+'.png',bbox_inches='tight')
    
    plt.close()

gif_filepath = './anim/ECMWF_PV_polarNH_'+date1+'-'+date2+'.gif'
make_animation()
IPdisplay.Image(url=gif_filepath)

In [ ]:
projection = ccrs.PlateCarree()
bounds = [(lonW, lonE, latS, latN)]

levels_z = np.arange(int(z.min()),int(z.max()),100)
wind_slice = slice(None, None, 10)

for i in range(len(time)):
    #print(date_str[i])
    fig = plt.figure(figsize=(15., 10.))
    ax = plt.subplot(111, projection=projection)
    ax.set_title('300hPa potential vorticity (PVU), geopotential height (mgp) and winds (kt)', loc='left')
    ax.set_title(date_str[i], loc='right')
    plot_background(ax)
    ax.set_extent(*bounds, crs=ccrs.PlateCarree())
    cs1 = ax.contour(lon, lat, z[i,lev300,:,:], levels_z, colors='black', transform=ccrs.PlateCarree())
    cs2 = ax.contourf(lon, lat, pv[i,lev300,:,:]*1e6, levs_pv, cmap=cmap,
                      extend='both', transform=ccrs.PlateCarree())
    cb = fig.colorbar(cs2, orientation='horizontal', aspect=65, shrink=0.5, pad=0.05, extendrect='True')
    cb.set_label('10$^{-6}$ K  m$^{2}$ kg$^{-1}$ s$^{-1}$')
    
    ax.barbs(lon[wind_slice], lat[wind_slice],
         u[i,lev300][wind_slice, wind_slice],
         v[i,lev300][wind_slice, wind_slice], pivot='middle',
         color='red', length=6.5, transform=ccrs.PlateCarree())

    figname='./anim/ECMWF_PV_Z_wind_'+date_str[i]
    plt.savefig(figname+'.png',bbox_inches='tight')
    
    plt.show()
    plt.close()

gif_filepath = './anim/ECMWF_PV_Z_wind_'+date1+'-'+date2+'.gif'
make_animation()
IPdisplay.Image(url=gif_filepath)

In [ ]:
date_selection='2014-01-26T00'

# Define cross section
lat1 = 55
lat2=lat1
lon1 = lonW
lon2 = lonE

# Find the lat/lon index values for the desired xsection
ilat1 = list(lat).index(lat1)
ilat2 = list(lat).index(lat2)
ilon1 = list(lon).index(lon1)
ilon2 = list(lon).index(lon2)

# Find the index value for the desired xsection time surface
itime = date_str.index(date_selection)

lon_profile1=2
lon_profile2=-13
lon_profile3=-33

# Fields
pv_xs= pv[itime,:,ilat1,ilon1:ilon2]
u_xs= u[itime,:,ilat1,ilon1:ilon2]
v_xs= v[itime,:,ilat1,ilon1:ilon2]
t_xs= t[itime,:,ilat1,ilon1:ilon2]
th_xs= theta[itime,:,ilat1,ilon1:ilon2]
the_xs= thetae[itime,:,ilat1,ilon1:ilon2]
lons_xs=lon[ilon1:ilon2]

levels_v = np.arange(-40,50,10)

# Plot
fig = plt.figure(1, figsize=(14, 12))
ax = plt.subplot(111)
fig.suptitle('Potential vorticity (PVU) potential temperature (K) and winds (kt)', fontsize=16)
ax.set_title(date_str[itime], loc='right', fontsize=14)
ax.set_title('Crossection at '+str(lat1)+'N', loc='left', fontsize=14)
ax.set_yscale('symlog')
plt.xlabel('Longitude')
plt.ylabel('Pressure level (hPa)')
ax.set_yticklabels(np.arange(1000, 0, -100))
ax.set_ylim(1000, 100)
ax.set_yticks(np.arange(1000, 0, -100))  
#ax.set_xticklabels(np.arange(-90, 90, 10))
#ax.set_xticks(lons_xs)    
cf = ax.contourf(lons_xs, lev, pv_xs*1e6,levs_pv2, cmap=cmap, extend='both')
cb = fig.colorbar(cf, orientation='horizontal', aspect=65, shrink=0.5, pad=0.05, extendrect='True')
c1 = ax.contour(lons_xs, lev, pv_xs*1e6, levs_15pvu, colors='black', linewidth=4)
c2 = ax.contour(lons_xs, lev, th_xs, levs_th, colors='grey')
ax.clabel(c2, fmt='%4.1i', fontsize=10)

#c3 = ax.contour(lons_xs, lev, v_xs, levels_v[levels_v != 0], colors='blue')
#ax.clabel(c3, fmt='%4.1i', fontsize=10)

wind = ax.barbs(lons_xs[10::18], lev, u_xs[:,10::18], v_xs[:,10::18], color='red')

plt.axvline(lon_profile1, color='black', linestyle="--")
plt.axvline(lon_profile2, color='black', linestyle="--")
plt.axvline(lon_profile3, color='black', linestyle="--")

# Plot small stamp map
iplev = list(lev).index(300)
ax_inset = fig.add_axes([0.925, 0.695, 0.2, 0.2], projection=ccrs.PlateCarree())
ax_inset.set_title('PV at 300hPa')
ax_inset.coastlines()
ax_inset.gridlines()
bounds = [(lonW, lonE, latS, latN)]
ax_inset.set_extent(*bounds, crs=ccrs.PlateCarree())
ax_inset.contour(lon, lat, pv2[itime,iplev,:,:]*1e6, levels=levs_pv, transform=ccrs.PlateCarree())
# Plot the path of the cross section
ax_inset.plot([lon1, lon2], [lat1, lat2], c='red', lw=1, transform=ccrs.PlateCarree())

figname='./figs/ECMWF_PV_XS_lat_'+str(lat1)+'_'+date2
plt.savefig(figname+'.png',bbox_inches='tight')

plt.show()

In [ ]:
ilon_profile1 = list(t_xs.longitude.values).index(lon_profile1)
t_profile1=t_xs[:,ilon_profile1]
th_profile1=th_xs[:,ilon_profile1]

ilon_profile2 = list(t_xs.longitude.values).index(lon_profile2)
t_profile2=t_xs[:,ilon_profile2]
th_profile2=th_xs[:,ilon_profile2]

ilon_profile3 = list(t_xs.longitude.values).index(lon_profile3)
t_profile3=t_xs[:,ilon_profile3]
th_profile3=th_xs[:,ilon_profile3]

print(t_profile1.shape)

fig = plt.figure(figsize=(15,8))
fig.suptitle('Temperature and potential temperature profiles', fontsize=16)

plt.subplot(121)
plt.title('Temperature profile')
plt.xlabel('Temperature (K)')
plt.ylabel('Pressure level (hPa)')
plt.yscale('log')
plt.gca().invert_yaxis()
plt.plot(t_profile1,lev, label='longitude '+str(lon_profile1))
plt.plot(t_profile2,lev, label='longitude '+str(lon_profile2))
plt.plot(t_profile3,lev, label='longitude '+str(lon_profile3))

plt.legend()

plt.subplot(122)
plt.title('Potential temperature profile')
plt.xlabel('Potential temperature (K)')
plt.ylabel('Pressure level (hPa)')
plt.yscale('log')
plt.gca().invert_yaxis()
plt.plot(th_profile1,lev, label='longitude '+str(lon_profile1))
plt.plot(th_profile2,lev, label='longitude '+str(lon_profile2))
plt.plot(th_profile3,lev, label='longitude '+str(lon_profile3))
plt.legend()

plt.show()

In [ ]:
# Plot
fig = plt.figure(1, figsize=(14, 12))
ax = plt.subplot(111)
fig.suptitle('Potential vorticity (PVU) equivalent potential temperature (K) and winds (kt)', fontsize=16)
ax.set_title(date_str[itime], loc='right', fontsize=14)
ax.set_title('Crossection at '+str(lat1)+'N', loc='left', fontsize=14)
ax.set_yscale('symlog')
plt.xlabel('Longitude')
plt.ylabel('Pressure level (hPa)')
ax.set_yticklabels(np.arange(1000, 0, -100))
ax.set_ylim(1000, 100)
ax.set_yticks(np.arange(1000, 0, -100))  
#ax.set_xticklabels(np.arange(-90, 90, 10))
#ax.set_xticks(lons_xs)    
cf = ax.contourf(lons_xs, lev, pv_xs*1e6,levs_pv2, cmap=cmap, extend='both')
c1 = ax.contour(lons_xs, lev, pv_xs*1e6, levs_15pvu, colors='black', linewidth=4)
c2 = ax.contour(lons_xs, lev, the_xs,levs_the, colors='grey')
ax.clabel(c2, fmt='%4.1i', fontsize=10)
cb = fig.colorbar(cf, orientation='horizontal', aspect=65, shrink=0.5, pad=0.05, extendrect='True')

wind = ax.barbs(lons_xs[10::18], lev, u_xs[:,10::18], v_xs[:,10::18], color='red')


# Plot small stamp map
iplev = list(lev).index(300)
ax_inset = fig.add_axes([0.925, 0.695, 0.2, 0.2], projection=ccrs.PlateCarree())
ax_inset.set_title('PV at 300hPa')
ax_inset.coastlines()
ax_inset.gridlines()
bounds = [(lonW, lonE, latS, latN)]
ax_inset.set_extent(*bounds, crs=ccrs.PlateCarree())
ax_inset.contour(lon, lat, pv2[itime,iplev,:,:]*1e6, levels=levs_pv, transform=ccrs.PlateCarree())
# Plot the path of the cross section
ax_inset.plot([lon1, lon2], [lat1, lat2], c='red', lw=1, transform=ccrs.PlateCarree())

figname='./figs/ECMWF_PV_XS2_lat_'+str(lat1)+'_'+date2
plt.savefig(figname+'.png',bbox_inches='tight')

plt.show()

In [ ]:
date_selection='2014-01-26T00'

# Define cross section
lat1 = latN
lat2 = 45
lon1 = -23
lon2=lon1

# Find the lat/lon index values for the desired xsection
ilat1 = list(lat).index(lat1)
ilat2 = list(lat).index(lat2)
ilon1 = list(lon).index(lon1)
ilon2 = list(lon).index(lon2)

# Find the time index value for the desired xsection
itime = date_str.index(date_selection)

# Fields
pv_xs= pv[itime,:,ilat1:ilat2,ilon1]
u_xs= u[itime,:,ilat1:ilat2,ilon1]
v_xs= v[itime,:,ilat1:ilat2,ilon1]
th_xs= theta[itime,:,ilat1:ilat2,ilon1]
the_xs= thetae[itime,:,ilat1:ilat2,ilon1]
lats_xs=lat[ilat1:ilat2]

print(lats_xs.shape)
print(u_xs.shape)
print(v_xs.shape)


# Plot
fig = plt.figure(1, figsize=(14, 12))
ax = plt.subplot(111)
fig.suptitle('Potential vorticity (PVU) and potential temperature (K)', fontsize=16)
ax.set_title(date_str[itime], loc='right', fontsize=14)
ax.set_title('Crossection at '+str(lon1)+'W', loc='left', fontsize=14)
ax.set_yscale('symlog')
plt.xlabel('Latitude')
plt.ylabel('Pressure level (hPa)')
ax.set_yticklabels(np.arange(1000, 0, -100))
ax.set_ylim(1000, 100)
ax.set_yticks(np.arange(1000, 0, -100))  
#ax.set_xticklabels(np.arange(-90, 90, 10))
#ax.set_xticks(lons_xs)    
cf = ax.contourf(lats_xs, lev, pv_xs*1e6,levs_pv2, cmap=cmap, extend='both')
c1 = ax.contour(lats_xs, lev, pv_xs*1e6,levs_15pvu, colors='black', linewidth=4)
c3 = ax.contour(lats_xs, lev, th_xs,levs_th, colors='grey')
ax.clabel(c3, fmt='%4.1i', fontsize=10)
cb = fig.colorbar(cf, orientation='horizontal', aspect=65, shrink=0.5, pad=0.05, extendrect='True')

wind = ax.barbs(lats_xs[2::8], lev, u_xs[:,2::8], v_xs[:,2::8], color='red')

# Plot small stamp map
iplev = list(lev).index(300)
ax_inset = fig.add_axes([0.925, 0.695, 0.2, 0.2], projection=ccrs.PlateCarree())
ax_inset.set_title('PV at 300hPa')
ax_inset.coastlines()
ax_inset.gridlines()
bounds = [(lonW, lonE, latS, latN)]
ax_inset.set_extent(*bounds, crs=ccrs.PlateCarree())
ax_inset.contour(lon, lat, pv2[itime,iplev,:,:]*1e6, levels=levs_pv, transform=ccrs.PlateCarree())
# Plot the path of the cross section
ax_inset.plot([lon1, lon2], [lat1, lat2], c='red', lw=1, transform=ccrs.PlateCarree())

figname='./figs/ECMWF_PV_XS_lon_'+str(lon1)+'_'+date2
plt.savefig(figname+'.png',bbox_inches='tight')

plt.show()

In [ ]:
# Plot
fig = plt.figure(1, figsize=(14, 12))
ax = plt.subplot(111)
fig.suptitle('Potential vorticity (PVU) and equivalent potential temperature (K)', fontsize=16)
ax.set_title(date_str[itime], loc='right', fontsize=14)
ax.set_title('Crossection at '+str(lon1)+'W', loc='left', fontsize=14)
ax.set_yscale('symlog')
plt.xlabel('Latitude')
plt.ylabel('Pressure level (hPa)')
ax.set_yticklabels(np.arange(1000, 0, -100))
ax.set_ylim(1000, 100)
ax.set_yticks(np.arange(1000, 0, -100))  
#ax.set_xticklabels(np.arange(-90, 90, 10))
#ax.set_xticks(lons_xs)    
cf = ax.contourf(lats_xs, lev, pv_xs*1e6,levs_pv2, cmap=cmap, extend='both')
c1 = ax.contour(lats_xs, lev, pv_xs*1e6,levs_15pvu, colors='black', linewidth=4)
c3 = ax.contour(lats_xs, lev, the_xs,levs_the, colors='grey')
ax.clabel(c3, fmt='%4.1i', fontsize=10)
cb = fig.colorbar(cf, orientation='horizontal', aspect=65, shrink=0.5, pad=0.05, extendrect='True')

wind = ax.barbs(lats_xs[2::8], lev, u_xs[:,2::8], v_xs[:,2::8], color='red')

# Plot small stamp map
iplev = list(lev).index(300)
ax_inset = fig.add_axes([0.925, 0.695, 0.2, 0.2], projection=ccrs.PlateCarree())
ax_inset.set_title('PV at 300hPa')
ax_inset.coastlines()
ax_inset.gridlines()
bounds = [(lonW, lonE, latS, latN)]
ax_inset.set_extent(*bounds, crs=ccrs.PlateCarree())
ax_inset.contour(lon, lat, pv2[itime,iplev,:,:]*1e6, levels=levs_pv, transform=ccrs.PlateCarree())
# Plot the path of the cross section
ax_inset.plot([lon1, lon2], [lat1, lat2], c='red', lw=1, transform=ccrs.PlateCarree())

figname='./figs/ECMWF_PV_XS2_lon_'+str(lon1)+'_'+date2
plt.savefig(figname+'.png',bbox_inches='tight')

plt.show()

# Tropopause dynamique

In [ ]:
f2pvu    = xr.open_dataset('./ecmwf/20140124-26_2pvu.nc').sel(time=slice(date1,date2))
print(f2pvu.variables)

In [ ]:
z0_2pvu = f2pvu['z']/g
theta0_2pvu = f2pvu['pt']
u0_2pvu = f2pvu['u']
v0_2pvu = f2pvu['v']

In [ ]:
z1_2pvu = lonflip(z0_2pvu)
theta1_2pvu = lonflip(theta0_2pvu)
u1_2pvu = lonflip(u0_2pvu)
v1_2pvu = lonflip(v0_2pvu)
lon0  = theta1_2pvu.longitude.values
print(lon0)

In [ ]:
latS=35
latN=65
lonW=-50
lonE=20

z_2pvu = z1_2pvu.sel(latitude=slice(latN,latS)).sel(longitude=slice(lonW,lonE))
theta_2pvu = theta1_2pvu.sel(latitude=slice(latN,latS)).sel(longitude=slice(lonW,lonE))
u_2pvu = u1_2pvu.sel(latitude=slice(latN,latS)).sel(longitude=slice(lonW,lonE))
v_2pvu = v1_2pvu.sel(latitude=slice(latN,latS)).sel(longitude=slice(lonW,lonE))

lat = theta_2pvu.latitude.values
lon  = theta_2pvu.longitude.values

print(z_2pvu.shape)
print(theta_2pvu.shape)


ddx_v2pvu=np.gradient(v_2pvu,axis=2)/dx
ddy_u2pvu=np.gradient(u_2pvu,axis=1)/dy
vort_2pvu=ddx_v2pvu-ddy_u2pvu+(u_2pvu/re)*np.tan(ylat*np.pi/180)

In [ ]:
levels_th2PVU = np.arange(280,395,5)
levels_z2PVU=np.arange(4500,10500,500)
wind_slice = slice(None, None, 5)

projection = ccrs.PlateCarree()
bounds = [(lonW, lonE, latS, latN)]

for i in range(len(time)):
    #print(date_str[i])
    fig = plt.figure(figsize=(17., 10.))
    ax = plt.subplot(111, projection=projection)
    ax.set_title('Geopopential height (mgp), potential temperature (K) and winds on the 2PVU surface', loc='left')
    ax.set_title(date_str[i], loc='right')
    plot_background(ax)
    ax.set_extent(*bounds, crs=ccrs.PlateCarree())
    cf = ax.contourf(lon, lat, theta_2pvu[i,:,:], levels_th2PVU, transform=ccrs.PlateCarree(), cmap='jet', extend='both')
    c3 = ax.contour(lon, lat, z_2pvu[i,:,:], levels_z2PVU, colors='brown', linewidths=1, transform=ccrs.PlateCarree())
    
    ax.barbs(lon[wind_slice], lat[wind_slice],
             u_2pvu[i][wind_slice, wind_slice],
             v_2pvu[i][wind_slice, wind_slice], pivot='middle',
             color='red', length=6.5, transform=ccrs.PlateCarree())
    
    ax.clabel(c3, inline=1, fmt='%4.1i', fontsize=10)
    h3,_ = c3.legend_elements()
    ax.legend([h3[0]], ['Z2PVU'], loc='lower right')
    
    cb = fig.colorbar(cf, orientation='horizontal', aspect=65, shrink=0.5, pad=0.05, extendrect='True')
    cb.set_label('K')
    
    figname='./anim/ECMWF_2PVU_'+date_str[i]
    plt.savefig(figname+'.png',bbox_inches='tight')
    
    plt.show()
    plt.close()
    
gif_filepath = './anim/ECMWF_2PVU_'+date1+'-'+date2+'.gif'
make_animation()
IPdisplay.Image(url=gif_filepath)

# Interaction barocline

In [ ]:
projection = ccrs.PlateCarree()
bounds = [(lonW, lonE, latS, latN)]

for i in range(len(time)):
    #print(date_str[i])
    fig = plt.figure(figsize=(17., 10.))
    ax = plt.subplot(111, projection=projection)
    ax.set_title('300hPa potential vorticity (PVU), 925hPa cyclonic relative vorticity (10$^{-5}$ s$^{-1}$) and 850hPa equivalent potential temperature (K)', loc='left')
    ax.set_title(date_str[i], loc='right')
    plot_background(ax)
    ax.set_extent(*bounds, crs=ccrs.PlateCarree())
    cf = ax.contourf(lon, lat, thetae[i,lev850,:,:], levels_the, transform=ccrs.PlateCarree(), cmap='jet', extend='both')
    c3 = ax.contour(lon, lat, vort[i,lev925,:,:]*10**5, levels_vor, colors='black', linewidths=1, transform=ccrs.PlateCarree())
    c4 = ax.contour(lon, lat, pv[i,lev300,:,:]*1e6, levs_pv3, colors='red', linewidths=1, transform=ccrs.PlateCarree())
    
    ax.clabel(c3, inline=1, fmt='%4.1i', fontsize=10)
    ax.clabel(c4, inline=1, fmt='%4.1i', fontsize=10)
    
    h3,_ = c3.legend_elements()
    h4,_ = c4.legend_elements()
    ax.legend([h3[0], h4[0]], ['vort925hPa', 'PV300hPa'], loc='lower right')
    
    cb = fig.colorbar(cf, orientation='horizontal', aspect=65, shrink=0.5, pad=0.05, extendrect='True')
    cb.set_label('K')
    
    figname='./anim/ECMWF_perturb9_'+date_str[i]
    plt.savefig(figname+'.png',bbox_inches='tight')
    
    plt.show()
    plt.close()
    
gif_filepath = './anim/ECMWF_perturb9_'+date1+'-'+date2+'.gif'
make_animation()
IPdisplay.Image(url=gif_filepath)

In [ ]:
projection = ccrs.PlateCarree()
bounds = [(lonW, lonE, latS, latN)]

for i in range(len(time)):
    #print(date_str[i])
    fig = plt.figure(figsize=(17., 10.))
    ax = plt.subplot(111, projection=projection)
    ax.set_title('2PVU geopotential height (mgp), 925hPa cyclonic relative vorticity (10$^{-5}$ s$^{-1}$) and 850hPa equivalent potential temperature (K)', loc='left')
    ax.set_title(date_str[i], loc='right')
    plot_background(ax)
    ax.set_extent(*bounds, crs=ccrs.PlateCarree())
    cf = ax.contourf(lon, lat, thetae[i,lev850,:,:], levels_the, transform=ccrs.PlateCarree(), cmap='jet', extend='both')
    c3 = ax.contour(lon, lat, vort[i,lev925,:,:]*10**5, levels_vor, colors='black', linewidths=1, transform=ccrs.PlateCarree())
    c4 = ax.contour(lon, lat, z_2pvu[i,:,:], levels_z2PVU, colors='brown', linewidths=1, transform=ccrs.PlateCarree())
    
    ax.clabel(c3, inline=1, fmt='%4.1i', fontsize=10)
    ax.clabel(c4, inline=1, fmt='%4.1i', fontsize=10)
    
    h3,_ = c3.legend_elements()
    h4,_ = c4.legend_elements()
    ax.legend([h3[0], h4[0]], ['vort925hPa', 'Z2PVU'], loc='lower right')
    
    cb = fig.colorbar(cf, orientation='horizontal', aspect=65, shrink=0.5, pad=0.05, extendrect='True')
    cb.set_label('K')
    
    figname='./anim/ECMWF_perturb10_'+date_str[i]
    plt.savefig(figname+'.png',bbox_inches='tight')
    
    plt.show()
    plt.close()
    
gif_filepath = './anim/ECMWF_perturb10_'+date1+'-'+date2+'.gif'
make_animation()
IPdisplay.Image(url=gif_filepath)

In [ ]:
for i in range(len(time)):
    
    plt.figure(figsize=(20, 20))
    ax = plt.axes(projection="3d", xlim=[lonW, lonE], ylim=[latS, latN], zlim=[0, 350])
    ax.set_title('PV at 300hPa, cyclonic relative vorticity at 925hPa and $\theta_e$ at 850hPa : '+date_str[i],
                 y=1.0, pad=-20, fontsize=14)    
    # Axes, grid
    plt.box(False)
    ax.set(frame_on=False)  # New
    ax.set_zticklabels([])
    ax.set_zticks([])
    ax.grid(False)
    ax.xaxis.pane.fill = False
    ax.yaxis.pane.fill = False
    ax.zaxis.pane.fill = False
    ax.xaxis.pane.set_edgecolor('w')
    ax.yaxis.pane.set_edgecolor('w')
    ax.zaxis.pane.set_edgecolor('w')

    # Background Map
    target_projection = ccrs.PlateCarree()
    feature = cfeature.NaturalEarthFeature("physical", "coastline", "10m")
    geoms = feature.geometries()
    geoms = [target_projection.project_geometry(geom, feature.crs) for geom in geoms]
    paths = list(itertools.chain.from_iterable(geos_to_path(geom) for geom in geoms))
    segments = []
    for path in paths:
        vertices = [vertex for vertex, _ in path.iter_segments()]
        vertices = np.asarray(vertices)
        segments.append(vertices)
    lc = LineCollection(segments, color="black", zorder=0)
    ax.add_collection3d(lc)
    
    # convert z to np array
    pv300 = np.array(pv[i,lev300,:,:]*1e6)
    th850 = np.array(thetae[i,lev850,:,:])
    vort925 = np.array(vort[i,lev925,:,:]*10**5)

    # Create meshgrid from lats, lons
    x = lon
    y = lat
    x2 = np.append(0, x.flatten())
    y2 = np.append(0, y.flatten())
    x2, y2 = np.meshgrid(lon, lat)
    
    # Create contour plots on level 0 of z axis for Thetae
    cf=ax.contourf(x2, y2, th850, levels_the, zdir="z", offset=0, cmap="jet",
                   extent='both', alpha=0.8, zorder=10, antialiased=False)
    ax.contour(x2, y2, vort925, levels_vor, zdir="z", offset=0, colors="k", linewidths=2, zorder=10)
    cbar2 = plt.colorbar(cf, shrink=0.5, pad=0.01, orientation="vertical", location="left")
    cbar2.set_label(label="K", fontsize=10)
    
    # Create contour plots on level 250 of z axis for PV300
    cf=ax.contourf(x2, y2, pv300, levs_pv, zdir="z", offset=250, cmap="viridis",
                   extent='both', alpha=1, zorder=100, antialiased=False)
    cbar2 = plt.colorbar(cf, shrink=0.5, pad=0.01, orientation="vertical")
    cbar2.set_label(label="PVU", fontsize=10)
    
    # Set view angle
    ax.view_init(25, 270)
        
    figname='./anim/ECMWF_PV300_Thetae850_vort925_'+date_str[i]
    plt.savefig(figname+'.png', transparent = False, bbox_inches = 'tight', pad_inches = 0)
    plt.close()

gif_filepath = './anim/ECMWF_PV300_Thetae850_vort925_'+date1+'-'+date2+'.gif'
make_animation()
IPdisplay.Image(url=gif_filepath)

In [ ]:
for i in range(len(time)):
    
    plt.figure(figsize=(20, 20))
    ax = plt.axes(projection="3d", xlim=[lonW, lonE], ylim=[latS, latN], zlim=[0, 350])
    ax.set_title('Geopotential height and $\theta$ at 2PVU, cyclonic relative vorticity at 925hPa and $\theta_e$ at 850hPa : '+date_str[i],
                 y=1.0, pad=-20, fontsize=14)    
    # Axes, grid
    plt.box(False)
    ax.set(frame_on=False)  # New
    ax.set_zticklabels([])
    ax.set_zticks([])
    ax.grid(False)
    ax.xaxis.pane.fill = False
    ax.yaxis.pane.fill = False
    ax.zaxis.pane.fill = False
    ax.xaxis.pane.set_edgecolor('w')
    ax.yaxis.pane.set_edgecolor('w')
    ax.zaxis.pane.set_edgecolor('w')

    # Background Map
    target_projection = ccrs.PlateCarree()
    feature = cfeature.NaturalEarthFeature("physical", "coastline", "10m")
    geoms = feature.geometries()
    geoms = [target_projection.project_geometry(geom, feature.crs) for geom in geoms]
    paths = list(itertools.chain.from_iterable(geos_to_path(geom) for geom in geoms))
    segments = []
    for path in paths:
        vertices = [vertex for vertex, _ in path.iter_segments()]
        vertices = np.asarray(vertices)
        segments.append(vertices)
    lc = LineCollection(segments, color="black", zorder=0)
    ax.add_collection3d(lc)

    # convert z to np array
    th_2pvu = np.array(theta_2pvu[i,:,:])
    alt_2pvu = np.array(z_2pvu[i,:,:])    
    th850 = np.array(thetae[i,lev850,:,:])
    vort925 = np.array(vort[i,lev925,:,:]*10**5)

    # Create meshgrid from lats, lons
    x = lon
    y = lat
    x2 = np.append(0, x.flatten())
    y2 = np.append(0, y.flatten())
    x2, y2 = np.meshgrid(lon, lat)
    
    # Create contour plots on level 0 of z axis for Thetae
    cf=ax.contourf(x2, y2, th850, levels_the, zdir="z", offset=0, cmap="jet",
                   extent='both', alpha=0.8, zorder=10, antialiased=False)
    ax.contour(x2, y2, vort925, levels_vor, zdir="z", offset=0, colors="k", linewidths=2, zorder=10)
    cbar2 = plt.colorbar(cf, shrink=0.5, pad=0.01, orientation="vertical", location="left")
    cbar2.set_label(label="K", fontsize=10)
    
    # Create contour plots on level 250 of z axis for Theta2PVU and Z2PVU
    cf=ax.contourf(x2, y2, th_2pvu, levels_th2PVU, zdir="z", offset=250, cmap="jet",
                   extent='both', alpha=0.8, zorder=100, antialiased=False)
    ax.contour(x2, y2, alt_2pvu, levels_z2PVU, zdir="z", offset=250, colors="k", linewidths=2, zorder=100)
    cbar2 = plt.colorbar(cf, shrink=0.5, pad=0.01, orientation="vertical")
    cbar2.set_label(label="K", fontsize=10)
    
    # Set view angle
    ax.view_init(25, 270)
        
    figname='./anim/ECMWF_2PVU_Thetae850_vort925_'+date_str[i]
    plt.savefig(figname+'.png', transparent = False, bbox_inches = 'tight', pad_inches = 0)
    plt.close()

gif_filepath = './anim/ECMWF_2PVU_Thetae850_vort925_'+date1+'-'+date2+'.gif'
make_animation()
IPdisplay.Image(url=gif_filepath)

# Tourbillon potentiel sur surface isentrope

In [ ]:
t = t0.sel(latitude=slice(90,30))
u = u0.sel(latitude=slice(90,30))
v = v0.sel(latitude=slice(90,30))
pv = pv0.sel(latitude=slice(90,30))

lat = pv.latitude.values
lon  = pv.longitude.values

https://unidata.github.io/MetPy/latest/api/generated/metpy.calc.isentropic_interpolation.html

In [ ]:
theta_lev=315
isen_level = np.array([theta_lev]) * units.kelvin

isen_press, isen_u, isen_v, isen_pv = mpcalc.isentropic_interpolation(isen_level, lev* units.hPa, t, u, v, pv)

In [ ]:
isen_press = isen_press.squeeze()
isen_u = isen_u.squeeze()
isen_v = isen_v.squeeze()
isen_pv = isen_pv.squeeze()

print(isen_pv.shape)

In [ ]:
projection=ccrs.NorthPolarStereo()
bounds = [(-180., 180., 30, 90)]

# Find the index value for the isobaric surface
iplev = list(lev).index(300)

levs_pv = np.arange(-1, 8.5, 0.5)

for i in range(len(time)):
    print(date_str[i])
    
    fig = plt.figure(figsize=(15., 10.))
    fig.suptitle('Potential vorticity : '+date_str[i], fontsize=16)
    
    ax = plt.subplot(121, projection=projection)
    ax.set_title('Potential vorticity (PVU) on the 300 hPa isobaric surface', loc='center')
    ax.set_extent(*bounds, ccrs.PlateCarree())
    ax.coastlines()
    # Compute a circle in axes coordinates, which we can use as a boundary
    # for the map. We can pan/zoom as much as we like - the boundary will be
    # permanently circular.
    angle = np.linspace(0, 2*np.pi, 100)
    center, radius = [0.5, 0.5], 0.5
    verts = np.vstack([np.sin(angle), np.cos(angle)]).T
    circle = mpath.Path(verts * radius + center)
    ax.set_boundary(circle, transform=ax.transAxes)
    ax.gridlines(crs=ccrs.PlateCarree(), linewidth=0.5, color='gray', alpha=0.5, linestyle='-')  
    cs1 = ax.contourf(lon, lat, pv[i,iplev,:,:]*1e6, levs_pv, cmap=cmap, extend='both', transform=ccrs.PlateCarree())
    cb = plt.colorbar(cs1, orientation='horizontal', aspect=65, shrink=0.5, pad=0.05, extendrect='True')
    cb.set_label('10$^{-6}$ K  m$^{2}$ kg$^{-1}$ s$^{-1}$')
    
    ax = plt.subplot(122, projection=projection)
    ax.set_title('Potential vorticity (PVU) on the '+str(theta_lev)+' isentropic surface', loc='center')
    ax.set_extent(*bounds, ccrs.PlateCarree())
    ax.coastlines()
    # Compute a circle in axes coordinates, which we can use as a boundary
    # for the map. We can pan/zoom as much as we like - the boundary will be
    # permanently circular.
    angle = np.linspace(0, 2*np.pi, 100)
    center, radius = [0.5, 0.5], 0.5
    verts = np.vstack([np.sin(angle), np.cos(angle)]).T
    circle = mpath.Path(verts * radius + center)
    ax.set_boundary(circle, transform=ax.transAxes)
    ax.gridlines(crs=ccrs.PlateCarree(), linewidth=0.5, color='gray', alpha=0.5, linestyle='-')  
    cs1 = ax.contourf(lon, lat, isen_pv[i,:,:]*1e6, levs_pv, cmap=cmap, extend='both', transform=ccrs.PlateCarree())
    cb = plt.colorbar(cs1, orientation='horizontal', aspect=65, shrink=0.5, pad=0.05, extendrect='True')
    cb.set_label('10$^{-6}$ K  m$^{2}$ kg$^{-1}$ s$^{-1}$')
    
    figname='./anim/ECMWF_PV_polarNH_isen_'+date_str[i]
    plt.savefig(figname+'.png',bbox_inches='tight')
    
    plt.close()

gif_filepath = './anim/ECMWF_PV_polarNH_isen_'+date1+'-'+date2+'.gif'
make_animation()
IPdisplay.Image(url=gif_filepath)

# Forçage géostrophique (Hoskins)

In [ ]:
latS=35
latN=65
lonW=-50
lonE=20

msl = msl1.sel(latitude=slice(latN,latS)).sel(longitude=slice(lonW,lonE))
z = z1.sel(latitude=slice(latN,latS)).sel(longitude=slice(lonW,lonE))
u = u1.sel(latitude=slice(latN,latS)).sel(longitude=slice(lonW,lonE))
v = v1.sel(latitude=slice(latN,latS)).sel(longitude=slice(lonW,lonE))
t = t1.sel(latitude=slice(latN,latS)).sel(longitude=slice(lonW,lonE))
w = w1.sel(latitude=slice(latN,latS)).sel(longitude=slice(lonW,lonE))
q = q1.sel(latitude=slice(latN,latS)).sel(longitude=slice(lonW,lonE))
pv = pv1.sel(latitude=slice(latN,latS)).sel(longitude=slice(lonW,lonE))

lat = z.latitude.values
lon  = w.longitude.values

print(z.shape)
print(msl.shape)

<div class="alert alert-danger">
$$\vec{Q} = (Q_1, Q_2)
=  - \frac{R}{\sigma p}\left(
         \frac{\partial \vec{v}_g}{\partial x} \cdot \nabla_p T,
         \frac{\partial \vec{v}_g}{\partial y} \cdot \nabla_p T
     \right)$$
</div>

In [ ]:
date_selection="2014-01-26T00"
itime = date_str.index(date_selection)
bounds = [(-30, -10, 50, 60)]
projection = ccrs.PlateCarree()

In [ ]:
z1=z[itime,lev700,:,:]
u1=u[itime,lev700,:,:]
v1=v[itime,lev700,:,:]
t1=t[itime,lev700,:,:]
w1=w[itime,lev700,:,:]
print(w1.shape)

z1 = gaussian_filter(z1, sigma=3.0)
u1 = gaussian_filter(u1, sigma=3.0)
v1 = gaussian_filter(v1, sigma=3.0)
t1 = gaussian_filter(t1, sigma=3.0)
w1 = gaussian_filter(w1, sigma=3.0)

ddx_z=np.gradient(z1,axis=1)/dx
ddy_z=np.gradient(z1,axis=0)/dy

ug=(-1)*(g/f)*ddy_z
vg=(g/f)*ddx_z

ddx_ug=np.gradient(ug,axis=1)/dx
ddx_vg=np.gradient(vg,axis=1)/dx
ddy_ug=np.gradient(ug,axis=0)/dy
ddy_vg=np.gradient(vg,axis=0)/dy

ddx_temp=np.gradient(t1,axis=1)/dx
ddy_temp=np.gradient(t1,axis=0)/dy

p=lev700*100
s=0.0001
q1=-(r/(s*p))*(ddx_temp*ddx_ug+ddy_temp*ddx_vg)
q2=-(r/(s*p))*(ddx_temp*ddy_ug+ddy_temp*ddy_vg)

ddx_q=np.gradient(q1,axis=1)/dx
ddy_q=np.gradient(q2,axis=0)/dy
divq=ddx_q+ddy_q
divq1 = gaussian_filter(divq, sigma=3.0)*10**11

print(divq1)

In [ ]:
wind_slice = slice(None, None, 1)

levels_divq = np.arange(-8,8.5,0.5)
levels_w = np.arange(-2,2.2,0.2)

fig = plt.figure(figsize=(15., 12.))
ax = fig.add_subplot(1, 1, 1, projection=projection)
ax.set_title('Geopotential height, Q vector divergence and vertical motion at 700 hPa', fontsize=14)
ax.set_title(date_str[itime], loc='right', fontsize=14)

plot_background(ax)
ax.set_extent(*bounds, crs=ccrs.PlateCarree())

cf = ax.contourf(lon, lat, divq1, levels_divq, transform=ccrs.PlateCarree(), cmap='bwr', extend='both')
cb = fig.colorbar(cf, orientation='horizontal', aspect=65, shrink=0.5, pad=0.05, extendrect='True')
cb.set_label('10$^{-13}$ s$^{-1}$')

c1 = ax.contour(lon, lat, z1, levels_z700, colors='black', linewidths=1, transform=ccrs.PlateCarree())
c2 = ax.contour(lon, lat, w1, levels_wpos[levels_wpos != 0], colors='brown', linewidths=1,
                transform=ccrs.PlateCarree()) 
c3 = ax.contour(lon, lat, w1, levels_wneg[levels_wneg != 0], colors='green', linewidths=1,
                linestyle="--", transform=ccrs.PlateCarree()) 

ax.clabel(c1, fmt='%4.1i', fontsize=10)
ax.clabel(c2, fmt='%4.1f', fontsize=10)
ax.clabel(c3, fmt='%4.1f', fontsize=10)

h1,_ = c1.legend_elements()
h2,_ = c2.legend_elements()
ax.legend([h1[0], h2[0], h3[0]], ['Z700hPa', 'VV+ 700hPa', 'VV- 700hPa'], loc='lower right')

ax.quiver(lon[wind_slice], lat[wind_slice],
         q1[wind_slice, wind_slice],
         q2[wind_slice, wind_slice], pivot='mid', headwidth=2, headlength=2,
         color='grey', scale_units='width', transform=ccrs.PlateCarree())

figname='./figs/ECMWF_Q_'+date_str[itime]

plt.savefig(figname+'.png',bbox_inches='tight')

plt.show()

# Diagnostics liés à la frontogenèse

In [ ]:
date_selection="2014-01-26T00"
itime = date_str.index(date_selection)
bounds = [(-40, 0, 40, 65)]

In [ ]:
ddx_v=np.gradient(v,axis=3)/dx
ddy_u=np.gradient(u,axis=2)/dy
ddx_u=np.gradient(u,axis=3)/dx
ddy_v=np.gradient(v,axis=2)/dy
ddx_theta=np.gradient(theta,axis=3)/dx
ddy_theta=np.gradient(theta,axis=2)/dy

<div class="alert alert-danger">
$$d_c = \frac{\partial v}{\partial x}+\frac{\partial u}{\partial y}$$
$$d_e = \frac{\partial u}{\partial x}-\frac{\partial v}{\partial y}$$
$$d = \sqrt{d_c^2+d_e^2}$$
</div>

In [ ]:
shrd = ddx_v + ddy_u
strd = ddx_u - ddy_v
tdef = np.sqrt(shrd**2 + strd**2)
print(strd.shape)

$$F=\frac{1}{2}\left|\nabla \theta\right|[d cos(2\beta)-div(\vec{V}_h)]$$

In [ ]:
mag_thta = np.sqrt(ddx_theta**2 + ddy_theta**2)

psi = 0.5 * np.arctan2(shrd, strd)
beta = np.arcsin((-ddx_theta * np.cos(psi) - ddy_theta * np.sin(psi)) / mag_thta)

frontogenesis1 = 0.5 * mag_thta * (tdef * np.cos(2 * beta) - div)
frontogenesis1=frontogenesis1*1000*100*3600*3

<div class="alert alert-danger">
$$F=\frac{1}{|\nabla \theta|}\left(\frac{-\partial \theta}{\partial x}\left(\frac{\partial u}{\partial x}\frac{\partial \theta}{\partial x}+\frac{\partial v}{\partial x}\frac{\partial \theta}{\partial y}\right)-\frac{\partial \theta}{\partial y}\left(\frac{\partial u}{\partial y}\frac{\partial \theta}{\partial x}+\frac{\partial v}{\partial y}\frac{\partial \theta}{\partial y}\right)\right)$$
</div>

In [ ]:
frontogenesis2 = (1 / mag_thta) * ((-ddx_theta)*(ddx_u*ddx_theta+ddx_v*ddy_theta)-
                                   ddy_theta*(ddy_u*ddx_theta+ddy_v*ddy_theta))
frontogenesis2=frontogenesis2*1000*100*3600*3

In [ ]:
levels_fronto = np.arange(-10,10.5,0.5)
levels_the2 = np.arange(250,345,2.5)
wind_slice = slice(None, None, 8)

fig = plt.figure(figsize=(15., 12.))
ax = fig.add_subplot(1, 1, 1, projection=projection)
ax.set_title('Geopotential height at 925 hPa, ThetaE and frontogenesis at 850 hPa', fontsize=14)
ax.set_title(date_str[itime], loc='right', fontsize=14)
plot_background(ax)
ax.set_extent(*bounds, crs=ccrs.PlateCarree())

cf = ax.contourf(lon, lat, frontogenesis1[itime,lev850,:,:], levels_fronto,
                 transform=ccrs.PlateCarree(), cmap='bwr', extend='both')
cb = fig.colorbar(cf, orientation='horizontal', aspect=65, shrink=0.5, pad=0.05, extendrect='True')

cb.set_label('K/100km/3h')

c1 = ax.contour(lon, lat, z[itime,lev925,:,:], levels_z925, colors='black',
                linewidths=1, transform=ccrs.PlateCarree())
c2 = ax.contour(lon, lat, thetae[itime,lev850,:,:], levels_the2, colors='blue',
                linewidths=1, transform=ccrs.PlateCarree())
ax.clabel(c1, fmt='%4.1i', fontsize=10)
ax.clabel(c2, fmt='%3.1i', fontsize=10)

ax.barbs(lon[wind_slice], lat[wind_slice],
u[itime,lev925][wind_slice, wind_slice],
v[itime,lev925][wind_slice, wind_slice], pivot='middle',
color='red', length=6.5, transform=ccrs.PlateCarree())

figname='./figs/ECMWF_fronto1_'+date_str[itime]

plt.savefig(figname+'.png',bbox_inches='tight')

plt.show()

In [ ]:
fig = plt.figure(figsize=(15., 12.))
ax = fig.add_subplot(1, 1, 1, projection=projection)
ax.set_title('Geopotential height at 925 hPa, ThetaE and frontogenesis at 850 hPa', fontsize=14)
ax.set_title(date_str[itime], loc='right', fontsize=14)
plot_background(ax)
ax.set_extent(*bounds, crs=ccrs.PlateCarree())

cf = ax.contourf(lon, lat, frontogenesis2[itime,lev850,:,:], levels_fronto,
                 transform=ccrs.PlateCarree(), cmap='bwr', extend='both')
cb = fig.colorbar(cf, orientation='horizontal', aspect=65, shrink=0.5, pad=0.05, extendrect='True')
cb.set_label('K/100km/3h')

c1 = ax.contour(lon, lat, z[itime,lev925,:,:], levels_z925, colors='black',
                linewidths=1, transform=ccrs.PlateCarree())
c2 = ax.contour(lon, lat, thetae[itime,lev850,:,:], levels_the2, colors='blue',
                linewidths=1, transform=ccrs.PlateCarree())
ax.clabel(c1, fmt='%4.1i', fontsize=10)
ax.clabel(c2, fmt='%3.1i', fontsize=10)

ax.barbs(lon[wind_slice], lat[wind_slice],
u[itime,lev925][wind_slice, wind_slice],
v[itime,lev925][wind_slice, wind_slice], pivot='middle',
color='red', length=6.5, transform=ccrs.PlateCarree())

figname='./figs/ECMWF_fronto2_'+date_str[itime]

plt.savefig(figname+'.png',bbox_inches='tight')

plt.show()

In [ ]:
wind_slice = slice(None, None, 4)

levels_the2 = np.arange(250,345,2.5)

fig = plt.figure(figsize=(15., 12.))
ax = fig.add_subplot(1, 1, 1, projection=projection)
ax.set_title('Geopotential height at 925 hPa, ThetaE and deformation at 850 hPa', fontsize=14)
ax.set_title(date_str[itime], loc='right', fontsize=14)
plot_background(ax)
ax.set_extent(*bounds, crs=ccrs.PlateCarree())

cf = ax.contourf(lon, lat, thetae[itime,lev850,:,:], levels_the, transform=ccrs.PlateCarree(),
                 cmap='jet', extend='both')
c = ax.contour(lon, lat, thetae[itime,lev850,:,:], levels_the, transform=ccrs.PlateCarree(),
               colors='grey', linewidths=1,)


cb = fig.colorbar(cf, orientation='horizontal', aspect=65, shrink=0.5, pad=0.05, extendrect='True')
cb.set_label('K')
c1 = ax.contour(lon, lat, z[itime,lev925,:,:], levels_z925, colors='black', linewidths=1,
                transform=ccrs.PlateCarree())
ax.clabel(c1, fmt='%4.1i', fontsize=10)

ax.quiver(lon[wind_slice], lat[wind_slice],
shrd[itime,lev850][wind_slice, wind_slice],
-strd[itime,lev850][wind_slice, wind_slice]+tdef[itime,lev850][wind_slice, wind_slice], pivot='mid',
color='red', scale=0.009, width=0.002, headwidth=4, transform=ccrs.PlateCarree())

ax.quiver(lon[wind_slice], lat[wind_slice],
-shrd[itime,lev850][wind_slice, wind_slice],
strd[itime,lev850][wind_slice, wind_slice]-tdef[itime,lev850][wind_slice, wind_slice], pivot='mid',
color='red', scale=0.009, width=0.002, headwidth=4, transform=ccrs.PlateCarree())

figname='./figs/ECMWF_fronto3_'+date_str[itime]

plt.savefig(figname+'.png',bbox_inches='tight')

plt.show()